In [1]:
# This produces the dataframe for TE

In [2]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates (not confirmed)
# unbalanced dataframe - pfr stats start at 2018; all other stats go back to 2017

In [3]:
## Notes: on the fantasypros data
# - I was unable to validate hash rows for the redzone data  

In [4]:
## Required installations
# 

In [5]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [6]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re
import time
from random import sample, uniform, seed
import io
from rapidfuzz import fuzz, process
import numpy as np
import hashlib
import shutil

In [7]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [8]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [9]:
### Begin: Python NFL Library Dataframe ###

In [10]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [11]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [12]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [13]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [14]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [15]:
## Output: a dataframe of NFL TE info and ids since 2017
# extract TE from the dataframe
# Create a new dataframe with only tight ends
tight_end_ids = all_players_id_data[all_players_id_data['position'] == 'TE']

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {tight_end_ids.shape}")

# Display the resulting dataframe for review
# display(tight_end_ids)

Shape of merged dataframe: (8782, 29)


In [16]:
## Output: a dataframe of NFL TE info, ids, and stats since 2017
# TE-specific columns (receiving-related)
te_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

# Pull TE-specific columns from weekly data
te_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + te_columns  # Include keys for merging
)

# Merge TE-specific stats with tight_end_ids
te_ids_weekly_stats = pd.merge(
    tight_end_ids,
    te_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {te_ids_weekly_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {te_ids_weekly_stats.shape[0] == tight_end_ids.shape[0]}"
)

# display the df
display(te_ids_weekly_stats)

# csv file
# te_ids_weekly_stats.to_csv('te_ids_weekly_stats.csv', index=False)

Downcasting floats.
Shape of merged dataframe: (8782, 44)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr
0,2017,REG,1,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,17.0,0,0.0,0.0,16.0,8.0,1.0,0.303061,0,1.062500,0.090909,0.080808,0.192929
1,2017,REG,2,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,4,11.0,1,0.0,0.0,28.0,0.0,1.0,0.946824,0,0.392857,0.102564,0.094276,0.219839
2,2017,REG,3,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,5,30.0,0,0.0,0.0,37.0,11.0,2.0,-5.984768,0,0.810811,0.128205,0.088517,0.254269
3,2017,REG,4,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,3,5,18.0,0,0.0,0.0,16.0,8.0,1.0,-0.309226,0,1.125000,0.135135,0.061303,0.245615
4,2017,REG,5,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,23.0,0,0.0,0.0,24.0,7.0,1.0,1.059868,0,0.958333,0.071429,0.086022,0.167358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,2024,REG,7,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,2,2,6.0,1,0.0,0.0,0.0,6.0,1.0,0.989268,0,0.000000,0.080000,0.000000,0.120000
8778,2024,REG,9,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,1,1,12.0,0,0.0,0.0,4.0,8.0,1.0,1.172226,0,3.000000,0.052632,0.033058,0.102088
8779,2024,REG,13,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,1,1,3.0,0,0.0,0.0,2.0,1.0,0.0,-0.259048,0,1.500000,0.033333,0.012048,0.058434
8780,2024,REG,15,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,1,1,7.0,0,0.0,0.0,3.0,4.0,0.0,-0.452960,0,2.333333,0.032258,0.011364,0.056342


In [17]:
# Output: imports the NFL next-generation stats (NGS) from the nfl python library

# Import the next-generation stats (NGS) from nfl.import_ngs_data()
# Note: NGS starts at week 0 (previous season totals) - not needed so drop those rows

# Pull NGS receiving data for the specified years
ngs_te_df = nfl.import_ngs_data('receiving', years)

# Exclude rows where 'week' == 0 and filter for 'TE' position in one step
ngs_te_df = ngs_te_df[(ngs_te_df['week'] != 0) & (ngs_te_df['player_position'] == 'TE')]

# Drop unnecessary columns (already in the nfl python baseline dataframe)
ngs_te_df = ngs_te_df.drop(columns=['season_type', 'player_position', 'receptions', 'targets', 'player_jersey_number'], errors='ignore')

# Display the resulting dataframe
print(f"Shape of NGS TE DataFrame after dropping columns: {ngs_te_df.shape}")
display(ngs_te_df)

# ***csv file ***
# ngs_te_df.to_csv('ngs_te_df.csv', index=False)

Shape of NGS TE DataFrame after dropping columns: (2480, 18)


,season,week,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name
1727,2017,1,Coby Fleener,NO,8.100000,1.639888,8.490000,19.676310,83.333333,54.0,1,2.162000,2.291386,-0.129386,00-0029697,Jacoby,Fleener,C.Fleener
1728,2017,1,Jared Cook,LV,7.838000,2.828689,6.724000,17.454055,100.000000,56.0,0,4.476000,5.092441,-0.616441,00-0027061,Jared,Cook,J.Cook
1730,2017,1,Jordan Reed,WAS,7.635000,3.704600,5.706250,12.343509,62.500000,36.0,0,2.638000,2.610629,0.027371,00-0030472,Jordan,Reed,J.Reed
1731,2017,1,Delanie Walker,TEN,7.572500,3.097148,10.901111,26.669023,77.777778,76.0,0,2.358571,3.025790,-0.667219,00-0024389,Delanie,Walker,D.Walker
1738,2017,1,Stephen Anderson,HOU,6.805000,3.975437,8.178000,12.325546,40.000000,14.0,0,5.225000,6.980167,-1.755167,00-0032725,Stephen,Anderson,S.Anderson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13310,2024,20,Tyler Higbee,LAR,4.483333,4.173928,2.690000,6.315147,70.000000,54.0,1,5.171429,7.902377,-2.730949,00-0033110,Tyler,Higbee,T.Higbee
13311,2024,20,Mark Andrews,BAL,4.423333,4.369371,8.424286,24.255512,71.428571,61.0,0,2.956000,3.607399,-0.651399,00-0034753,Mark,Andrews,M.Andrews
13315,2024,21,Dallas Goedert,PHI,7.668750,5.102511,5.158750,29.647989,87.500000,85.0,0,8.345714,7.087443,1.258271,00-0034351,Dallas,Goedert,D.Goedert
13319,2024,21,Zach Ertz,WAS,5.183571,4.100899,6.581875,32.005227,68.750000,104.0,0,2.555455,2.918727,-0.363273,00-0030061,Zachary,Ertz,Z.Ertz


In [18]:
# Output: a dataframe of NFL TE info, ids, weekly stats, and next-gen stats since 2017

# Joins te_ids_weekly_stats dataframe with ngs_te_df using the keys 'player_id' and 'player_gsis_id'
# Merge te_ids_weekly_stats with ngs_te_df using a left join
te_ids_ngs_weekly_stats = pd.merge(
    te_ids_weekly_stats,
    ngs_te_df,
    left_on=['player_id', 'season', 'week'],  # Keys from te_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_te_df
    how='left'  # Retain all rows from te_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {te_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {te_ids_weekly_stats.shape[0] == te_ids_ngs_weekly_stats.shape[0]}")

# Display a sample of the merged dataframe
display(te_ids_ngs_weekly_stats)

# ***csv file***
# te_ids_ngs_weekly_stats.to_csv('te_ids_ngs_weekly_stats.csv', index=False)

Shape of merged dataframe: (8782, 60)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name
0,2017,REG,1,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,17.0,0,0.0,0.0,16.0,8.0,1.0,0.303061,0,1.062500,0.090909,0.080808,0.192929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,4,11.0,1,0.0,0.0,28.0,0.0,1.0,0.946824,0,0.392857,0.102564,0.094276,0.219839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,5,30.0,0,0.0,0.0,37.0,11.0,2.0,-5.984768,0,0.810811,0.128205,0.088517,0.254269,Antonio Gates,LAC,3.356,1.639471,7.546,8.936735,40.0,30.0,0.0,8.005,9.593124,-1.588124,00-0021547,Antonio,Gates,A.Gates
3,2017,REG,4,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,3,5,18.0,0,0.0,0.0,16.0,8.0,1.0,-0.309226,0,1.125000,0.135135,0.061303,0.245615,Antonio Gates,LAC,5.410,2.747702,4.178,7.782000,60.0,18.0,0.0,2.480,1.649501,0.830499,00-0021547,Antonio,Gates,A.Gates
4,2017,REG,5,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,23.0,0,0.0,0.0,24.0,7.0,1.0,1.059868,0,0.958333,0.071429,0.086022,0.167358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,2024,REG,7,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,2,2,6.0,1,0.0,0.0,0.0,6.0,1.0,0.989268,0,0.000000,0.080000,0.000000,0.120000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8778,2024,REG,9,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,1,1,12.0,0,0.0,0.0,4.0,8.0,1.0,1.172226,0,3.000000,0.052632,0.033058,0.102088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8779,2024,REG,13,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0

In [19]:
# Output: pro-football reference dataframe for receiving data from the python nfl library
# note: PFR data not available before 2018
# there is no position info so the data will pull WR, TE, and RB receiving data

# Define the range of years for PFR data (2018 to the current year)
pfr_years = list(range(2018, current_year))

# import pro-football reference data
pfr_rec_df = nfl.import_weekly_pfr('rec',pfr_years)

# Drop unnecessary columns
pfr_rec_df = pfr_rec_df.drop(
    columns=['game_id','pfr_game_id','receiving_int','rushing_broken_tackles', 
             'passing_drops', 'passing_drop_pct'], errors='ignore')

# display dataframe
print(f"Shape of PFR dataframe: {pfr_rec_df.shape}")
display(pfr_rec_df)

# *** csv file *** 
# pfr_rec_df.to_csv('pfr_rec_df.csv', index=False)

Shape of PFR dataframe: (31191, 11)


,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
0,2018,1,REG,PHI,ATL,Nelson Agholor,AghoNe00,0.0,0.0,0.000,80.4
1,2018,1,REG,PHI,ATL,Zach Ertz,ErtzZa00,0.0,2.0,0.200,63.7
2,2018,1,REG,PHI,ATL,Darren Sproles,SproDa00,1.0,1.0,0.143,62.8
3,2018,1,REG,PHI,ATL,DeAndre Carter,CartDe02,0.0,0.0,0.000,108.3
4,2018,1,REG,PHI,ATL,Nick Foles,FoleNi00,0.0,0.0,0.000,118.7
...,...,...,...,...,...,...,...,...,...,...,...
4448,2024,22,SB,KC,PHI,JuJu Smith-Schuster,SmitJu00,0.0,0.0,0.000,100.0
4449,2024,22,SB,KC,PHI,Noah Gray,GrayNo00,0.0,0.0,0.000,79.2
4450,2024,22,SB,KC,PHI,Kareem Hunt,HuntKa00,0.0,0.0,0.000,87.5
4451,2024,22,SB,KC,PHI,Isiah Pacheco,PachIs00,0.0,0.0,0.000,56.2


In [20]:
# Output: a dataframe of NFL TE info, ids, weekly stats, next-gen stats, and pro-football reference data
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Merge the pfr_rec_df with the te_ids_ngs_weekly_stats dataframe
# Merge the dataframes with a LEFT JOIN
te_ids_ngs_pfr_stats = pd.merge(
    te_ids_ngs_weekly_stats,
    pfr_rec_df,  # Use the full PFR dataframe as position data is unavailable
    left_on=['pfr_id', 'season', 'week'],  # Keys from te_ids_ngs_weekly_stats
    right_on=['pfr_player_id', 'season', 'week'],  # Keys from pfr_rec_df
    how='left'  # Retain all rows from te_ids_ngs_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {te_ids_ngs_pfr_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {te_ids_weekly_stats.shape[0] == te_ids_ngs_weekly_stats.shape[0] == te_ids_ngs_pfr_stats.shape[0]}"
)

# Display the first few rows of the merged dataframe for review
display(te_ids_ngs_pfr_stats)

# *** csv file ***
# te_ids_ngs_pfr_stats.to_csv('te_ids_ngs_pfr_stats.csv', index=False)

Shape of merged dataframe: (8782, 69)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
0,2017,REG,1,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,17.0,0,0.0,0.0,16.0,8.0,1.0,0.303061,0,1.062500,0.090909,0.080808,0.192929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,4,11.0,1,0.0,0.0,28.0,0.0,1.0,0.946824,0,0.392857,0.102564,0.094276,0.219839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,5,30.0,0,0.0,0.0,37.0,11.0,2.0,-5.984768,0,0.810811,0.128205,0.088517,0.254269,Antonio Gates,LAC,3.356,1.639471,7.546,8.936735,40.0,30.0,0.0,8.005,9.593124,-1.588124,00-0021547,Antonio,Gates,A.Gates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,3,5,18.0,0,0.0,0.0,16.0,8.0,1.0,-0.309226,0,1.125000,0.135135,0.061303,0.245615,Antonio Gates,LAC,5.410,2.747702,4.178,7.782000,60.0,18.0,0.0,2.480,1.649501,0.830499,00-0021547,Antonio,Gates,A.Gates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0021547,A.Gates,TE,TE,LAC,7236,82da09c2-e542-4f7d-87d9-24dfd8e014de,9027.0,1609.0,322.0,antoniogates/2505299,5362.0,6663.0,NaN,396811.0,NaN,NaN,3172.0,611.0,NaN,6663.0,234249.0,7884.0,NaN,Antonio Gates,antonio gates,2,3,23.0,0,0.0,0.0,24.0,7.0,1.0,1.059868,0,0.958333,0.071429,0.086022,0.167358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,2024,REG,7,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,1615.0,40931.0,0.0,NaN,1228920.0,Ben Sinnott,ben sinnott,2,2,6.0,1,0.0,0.0,0.0,6.0,1.0,0.989268,0,0.000000,0.080000,0.000000,0.120000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REG,WAS,CAR,Ben Sinnott,SinnBe00,0.0,0.0,0.0,118.7
8778,2024,REG,9,00-0039912,B.Sinnott,TE,TE,WAS,16644,b5c717af-9116-483c-9ab3-c9db0a988ab8,26310.0,NaN,11596.0,NaN,4690923.0,40931.0,NaN,3168647.0,SinnBe00,ben-sinnott-1,17833.0,NaN,161

In [21]:
# *** Baseline Dataframe ***

# Output: a dataframe of NFL TE info, ids, weekly stats, next-gen stats, and pro-football reference data
# Output: Ordered the df by year, week, and receiving yards
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Order the dataframe by season (year), week, and receiving_yards
te_ids_ngs_pfr_stats_sorted = te_ids_ngs_pfr_stats.sort_values(
    by=['season', 'week', 'receiving_yards'], 
    ascending=[True, True, False]  # Ascending for season and week, descending for receiving_yards
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {te_ids_ngs_pfr_stats_sorted.shape}")

# Row integrity check
print(
    f"Row count matches: {te_ids_weekly_stats.shape[0] == te_ids_ngs_weekly_stats.shape[0] == te_ids_ngs_pfr_stats.shape[0] == te_ids_ngs_pfr_stats_sorted.shape[0]}"
)

# Display the sorted dataframe
print("Dataframe sorted by season, week, and receiving_yards:")
display(te_ids_ngs_pfr_stats_sorted)

# *** csv file ***
# te_ids_ngs_pfr_stats_sorted.to_csv('te_ids_ngs_pfr_stats_sorted.csv', index=False)

Shape of merged dataframe: (8782, 69)
Row count matches: True
Dataframe sorted by season, week, and receiving_yards:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
833,2017,REG,1,00-0032392,A.Hooper,TE,TE,ATL,12677,90c2a93f-d837-4e1b-b57c-56648903a8db,15581.0,10715.0,3202.0,NaN,3043275.0,29315.0,NaN,2067004.0,HoopAu00,austin-hooper-1,10748.0,11301.0,NaN,29315.0,739424.0,17963.0,739424.0,Austin Hooper,austin hooper,2,2,128.0,1,0.0,0.0,45.0,83.0,2.0,11.335708,0,2.844445,0.066667,0.208333,0.245833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,2017,REG,1,00-0030061,Z.Ertz,TE,TE,PHI,11247,de3421f7-2147-4835-89a5-724e87bad463,11689.0,7816.0,1339.0,zachertz/2540158,15835.0,26658.0,NaN,1685963.0,ErtzZa00,zach-ertz-1,8781.0,8409.0,305.0,26658.0,503177.0,14856.0,503177.0,Zach Ertz,zach ertz,8,8,93.0,0,0.0,0.0,78.0,15.0,6.0,7.477927,0,1.192308,0.210526,0.194514,0.451949,Zach Ertz,PHI,4.5000,2.710968,9.355000,18.126332,100.000000,93.0,0.0,2.270000,2.177001,0.092999,00-0030061,Zachary,Ertz,Z.Ertz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,2017,REG,1,00-0024389,De.Walker,TE,TE,TEN,8416,ccce5e8e-52ca-4f0f-a40f-fe5e7227d156,9172.0,3120.0,284.0,delaniewalker/2495966,9761.0,7924.0,4353.0,1109396.0,WalkHu00,NaN,4888.0,3976.0,NaN,7924.0,218943.0,7175.0,NaN,Delanie Walker,delanie walker,7,9,76.0,0,0.0,0.0,104.0,12.0,5.0,5.525133,0,0.730769,0.219512,0.279570,0.524967,Delanie Walker,TEN,7.5725,3.097148,10.901111,26.669023,77.777778,76.0,0.0,2.358571,3.025790,-0.667219,00-0024389,Delanie,Walker,D.Walker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
592,2017,REG,1,00-0031299,J.Matthews,TE,TE,BUF,11676,7b96a836-666b-47b6-a0a7-9dbb0b4c53e8,12124.0,8677.0,1800.0,jordanmatthews/2543500,16763.0,27570.0,NaN,1759816.0,MattJo00,jordan-matthews-1,9273.0,9420.0,NaN,27570.0,555648.0,15974.0,555648.0,Jordan Matthews,jordan matthews,2,3,61.0,0,0.0,0.0,56.0,42.0,2.0,5.050018,0,1.089286,0.111111,0.306011,0.380874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2017,REG,1,00-0022127,J.Witten,TE,TE,DAL,6997,e38c9b1b-7c51-48a2-ac1d-a752502e8930,9275.0,1384.0,23.0,jasonwitten/2505629,4527.0,6405.0,NaN,396134.0,WittJa00,jason-witten-1,3086.0,1990.0,NaN,6405.0,184571.0,722.0,NaN,Jason Witten,jason witten,7,9,59.0,1,0.0,0.0,61.0,12.0,1.0,2.561272,0,0.967213,0.236842,0.205387,0.499034,Jason Witten,DAL,4.0820,1.552612,6.372222,18.549065,77.777778,59.0,1.0,1.725714,1.558290,0.167425,00-0022127,Christopher,Witten,J.Witten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8222,2024,POST,21,00-0036628,J.Bates,TE,TE,WAS,15344,3208880b-949f-4926-85eb-1940770550ea,22833.0,42275.0,7716.0,NaN,4048228.0,33512.0,NaN,2257291.0,BateJo00,john-bates-1,15289.0,NaN,NaN,33512.0,0.0,NaN,944038.0,John Bates,john bates,2,2,11.0,0,0.0,0.0,9.0,2.0,1.0,-0.068331,0,1.222222,0.042553,0.025641,0.081779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON,WAS,PHI,John Bat

In [22]:
### End: Python NFL Library Dataframe ###

In [23]:
### Begin: Python NFL Library Data Integrity Checks ###

In [24]:
## Data Integrity Checks ##
# load the datframe from the variables in memory or csv files in the current working directory
def load_dataframe(var_name, file_name):
    try:
        return globals()[var_name]  # Try to get the variable from the global namespace
    except KeyError:
        print(f"⚠️ {var_name} not found in memory. Reading from {file_name}")
        return pd.read_csv(file_name)

In [25]:
def test_te_weekly_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on TE Weekly Stats data by comparing merged and unmerged dataframes.
    This function checks consistency across selected numerical columns.

    Parameters:
    years (range): Range of years to test.
    num_samples (int): Number of random samples per year.
    tolerance (float): Allowed numerical difference for matching values.

    Returns:
    None
    """
    print("\n🔹 Running TE Weekly Stats Integrity Check...")

    # Load DataFrames
    merged_df = load_dataframe("te_ids_ngs_pfr_stats_sorted", "te_ids_ngs_pfr_stats_sorted.csv")
    weekly_df = load_dataframe("te_ids_weekly_stats", "te_ids_weekly_stats.csv")

    print(f"✅ Using {'in-memory variable' if 'te_ids_ngs_pfr_stats_sorted' in globals() else 'CSV file'}: te_ids_ngs_pfr_stats_sorted")
    print(f"✅ Using {'in-memory variable' if 'te_ids_weekly_stats' in globals() else 'CSV file'}: te_ids_weekly_stats")

    mismatches = []

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data for the specified year
        merged_year_df = merged_df[merged_df["season"] == year]
        weekly_year_df = weekly_df[weekly_df["season"] == year]

        # Ensure numeric columns
        numeric_cols = ["receptions", "targets", "receiving_yards", "receiving_air_yards", "receiving_yards_after_catch", "receiving_first_downs"]
        merged_year_df.loc[:, numeric_cols] = merged_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")
        weekly_year_df.loc[:, numeric_cols] = weekly_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

        # Select random sample of players for this year
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            weekly_player = weekly_year_df[weekly_year_df["player_id"] == player_id]

            # Loop through each week and compare values
            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                weekly_row = weekly_player[weekly_player["week"] == week]

                if merged_row.empty or weekly_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare numerical columns
                for col in numeric_cols:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    weekly_val = weekly_row[col].values[0] if col in weekly_row else None

                    if merged_val is not None and weekly_val is not None:
                        if abs(float(merged_val) - float(weekly_val)) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, weekly_val))

        print(f"Tested {len(sampled_players)} players in Year {year}.")

    # Summary of mismatches
    print(f"\nTotal mismatches found in TE Weekly Stats: {len(mismatches)}")
    if mismatches:
        print("\nMismatch Details (Year, Player ID, Week, Column, Merged Value, Weekly Value):")
        for m in mismatches[:10]:  # Print only first 10 mismatches
            print(m)
    else:
        print("✅ TE Weekly Stats Integrity Check Passed!")

# Run the test
test_te_weekly_stats()



🔹 Running TE Weekly Stats Integrity Check...
✅ Using in-memory variable: te_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: te_ids_weekly_stats

Testing Year: 2017
Tested 25 players in Year 2017.

Testing Year: 2018
Tested 25 players in Year 2018.

Testing Year: 2019
Tested 25 players in Year 2019.

Testing Year: 2020
Tested 25 players in Year 2020.

Testing Year: 2021
Tested 25 players in Year 2021.

Testing Year: 2022
Tested 25 players in Year 2022.

Testing Year: 2023
Tested 25 players in Year 2023.

Total mismatches found in TE Weekly Stats: 0
✅ TE Weekly Stats Integrity Check Passed!


In [26]:
def test_te_ngs_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on TE Next-Gen Stats (NGS) data.
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running TE Next-Gen Stats (NGS) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("te_ids_ngs_pfr_stats_sorted", "te_ids_ngs_pfr_stats_sorted.csv")
    ngs_df = load_dataframe("ngs_te_df", "ngs_te_df.csv")

    # Validation message
    print("✅ Using in-memory variable: te_ids_ngs_pfr_stats_sorted" if "te_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded te_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: ngs_te_df" if "ngs_te_df" in globals() else "✅ Loaded ngs_te_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year]
        ngs_year_df = ngs_df[ngs_df["season"] == year]

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            ngs_player = ngs_year_df[ngs_year_df["player_gsis_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                ngs_row = ngs_player[ngs_player["week"] == week]

                if merged_row.empty or ngs_row.empty:
                    continue  # Skip if there's no data for this player/week
                
                # Compare Next-Gen Stats columns
                for col in [
                    "avg_cushion", "avg_separation", "avg_intended_air_yards",
                    "percent_share_of_intended_air_yards", "catch_percentage",
                    "avg_yac", "avg_expected_yac", "avg_yac_above_expectation"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    ngs_val = ngs_row[col].values[0] if col in ngs_row else None

                    if merged_val is not None and ngs_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        ngs_val = float(ngs_val)  # Ensure numeric conversion

                        if abs(merged_val - ngs_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, ngs_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in TE Next-Gen Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, NGS Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ TE Next-Gen Stats Integrity Check Passed!")

# Run the test
test_te_ngs_stats()



🔹 Running TE Next-Gen Stats (NGS) Integrity Check...
✅ Using in-memory variable: te_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: ngs_te_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in TE Next-Gen Stats: 0
✅ TE Next-Gen Stats Integrity Check Passed!


In [27]:
def test_te_pfr_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on TE data from Pro-Football Reference (PFR).
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running TE Pro-Football Reference (PFR) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("te_ids_ngs_pfr_stats_sorted", "te_ids_ngs_pfr_stats_sorted.csv")
    pfr_df = load_dataframe("pfr_rec_df", "pfr_rec_df.csv")

    # Validation message
    print("✅ Using in-memory variable: te_ids_ngs_pfr_stats_sorted" if "te_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded te_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: pfr_rec_df" if "pfr_rec_df" in globals() else "✅ Loaded pfr_rec_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year].copy()
        pfr_year_df = pfr_df[pfr_df["season"] == year].copy()

        # Ensure 'pfr_id' and 'pfr_player_id' are treated as strings
        merged_year_df["pfr_id"] = merged_year_df["pfr_id"].astype(str)
        pfr_year_df["pfr_player_id"] = pfr_year_df["pfr_player_id"].astype(str)

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["pfr_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["pfr_id"] == player_id]
            pfr_player = pfr_year_df[pfr_year_df["pfr_player_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                pfr_row = pfr_player[pfr_player["week"] == week]

                if merged_row.empty or pfr_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare PFR Stats columns
                for col in [
                    "receiving_broken_tackles", "receiving_drop", "receiving_drop_pct", "receiving_rat"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    pfr_val = pfr_row[col].values[0] if col in pfr_row else None

                    if merged_val is not None and pfr_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        pfr_val = float(pfr_val)  # Ensure numeric conversion

                        if abs(merged_val - pfr_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, pfr_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in TE PFR Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, PFR Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ TE Pro-Football Reference (PFR) Integrity Check Passed!")

# Run the test
test_te_pfr_stats()



🔹 Running TE Pro-Football Reference (PFR) Integrity Check...
✅ Using in-memory variable: te_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: pfr_rec_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in TE PFR Stats: 0
✅ TE Pro-Football Reference (PFR) Integrity Check Passed!


In [28]:
### End: Python NFL Library Data Integrity Checks ###

In [29]:
### Begin:fantasypros webscraping ###

In [30]:
# Generates a list of (year, week) combinations for web scraping.
# - 2017-2020: Weeks 1-17
# - 2021 and beyond: Weeks 1-18
def generate_year_week_combinations(start_year, end_year):
    year_week_combinations = []
    for year in range(start_year, end_year + 1):
        max_week = 17 if year <= 2020 else 18
        year_week_combinations.extend([(year, week) for week in range(1, max_week + 1)])
    return year_week_combinations

In [31]:
# Output: a dataframe of weekly TE fantasypros advanced stats week-by-week for defined years
# Define the function to scrape weekly TE advanced stats from Fantasy Pros

def scrape_fantasypros_te_all_weeks_years(start_year, end_year):
    """
    Scrapes FantasyPros TE advanced stats for all weeks and years with correct ID extraction.
    """
    year_week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in year_week_combinations:
        try:
            # Construct the URL
            url = f"https://www.fantasypros.com/nfl/advanced-stats-te.php?year={year}&week={week}&range=week&view=pergame"
            response = requests.get(url)
            response.raise_for_status()

            # Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract table headers
            table_headers = [header.text.strip() for header in soup.find('thead').find_all('th')]

            # Find table rows
            table_rows = soup.find('tbody').find_all('tr')

            for row in table_rows:
                # Extract FantasyPros Player ID correctly
                fantasypros_id = "Unknown"
                player_name = "Unknown"
                
                player_link = row.find("a", class_="fp-player-link")
                if player_link:
                    class_list = player_link.get("class", [])
                    for class_name in class_list:
                        if class_name.startswith("fp-id-"):
                            fantasypros_id = class_name.replace("fp-id-", "")
                            break
                    player_name = player_link.text.strip()

                # Extract the rest of the row data
                row_data = [cell.text.strip() for cell in row.find_all('td')]

                if len(row_data) == len(table_headers):
                    all_data.append([year, week, fantasypros_id, player_name] + row_data)

            time.sleep(uniform(0.3, 0.9))  # Respectful scraping

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    # Convert to DataFrame
    column_names = ['Year', 'Week', 'FantasyPros_ID', 'Player'] + table_headers
    te_fp_advanced_stats_df = pd.DataFrame(all_data, columns=column_names)
    te_fp_advanced_stats_df.drop(columns=['Rank'], inplace=True, errors='ignore')

    return te_fp_advanced_stats_df

# ✅ Run the full scrape
te_fp_advanced_stats_df = scrape_fantasypros_te_all_weeks_years(2017, years[-1])

# ✅ Display dataset shape for verification
print(f"\n📊 **Shape of FantasyPros TE advanced DataFrame:** {te_fp_advanced_stats_df.shape}")

# ✅ Display first few rows to confirm FantasyPros IDs are correctly extracted
display(te_fp_advanced_stats_df.head(10))

# *** csv file ***
# te_fp_advanced_stats_df.to_csv("fantasypros_te_advanced_stats.csv", index=False)


📊 **Shape of FantasyPros TE advanced DataFrame:** (1120, 22)


,Year,Week,FantasyPros_ID,Player,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS
0,2017,1,15581,Austin Hooper,Austin Hooper (NE),1,2,128,45,0,83,65,3,2,2,0,0,2,2,2,2,1
1,2017,1,13959,Jesse James,Jesse James (FA),1,6,41,15,0,26,7,0,8,6,0,2,1,0,0,0,0
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0
6,2017,1,9906,Kyle Rudolph,Kyle Rudolph (FA),1,3,26,23,0,3,3,0,3,3,0,1,1,0,0,0,0
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0
8,2017,2,9721,Rob Gronkowski,Rob Gronkowski (FA),1,6,116,56,0,60,25,1,9,8,2,1,5,2,1,1,1
9,2017,2,11594,Travis Kelce,Travis Kelce (KC),1,8,103,39,0,64,33,0,10,8,0,1,4,1,1,1,0


In [32]:
# Output: a dataframe of TE fantasy points and % rostered stats week-by-week for defined years

def te_scrape_fantasypros_fpts_rost(start_year, end_year):
    """
    Scrapes FantasyPros TE FPTS and % Rostered data for all weeks and years, including FantasyPros IDs.
    """
    week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in week_combinations:
        try:
            # Build the URL for TE
            url = f"https://www.fantasypros.com/nfl/stats/te.php?year={year}&week={week}&range=week"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)
                df = pd.read_html(io.StringIO(table_str), header=1)[0]

                # Extract IDs and names
                table_rows = table.find("tbody").find_all("tr")
                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"

                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        class_list = player_link.get("class", [])
                        for class_name in class_list:
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")
                                break
                        player_name = player_link.text.strip()

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names
                df["Year"] = year
                df["Week"] = week

                all_data.append(df)

            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    te_fpts_perct_rost_df = pd.concat(all_data, ignore_index=True)

    columns_to_drop = [
        "Rank", "REC", "TGT", "YDS", "Y/R", "LG", "20+", "TD", 
        "ATT", "YDS.1", "TD.1", "FL", "G", "FPTS/G"
    ]
    te_fpts_perct_rost_df.drop(columns=columns_to_drop, errors="ignore", inplace=True)

    # *** csv file ***
    # te_fpts_perct_rost_df.to_csv("fantasypros_te_fpts_perct_rost.csv", index=False)

    print(f"\n📊 **Shape of TE FPTS and % Rostered dataframe after column removal:** {te_fpts_perct_rost_df.shape}")
    display(te_fpts_perct_rost_df.head(10))

    return te_fpts_perct_rost_df

# ✅ Run the scraping function for all years
te_fpts_perct_rost_df = te_scrape_fantasypros_fpts_rost(2017, years[-1])



📊 **Shape of TE FPTS and % Rostered dataframe after column removal:** (15142, 6)


,FantasyPros_ID,Player,FPTS,ROST,Year,Week
0,15581,Austin Hooper,18.8,3.5%,2017,1
1,13959,Jesse James,16.1,0.0%,2017,1
2,9275,Jason Witten,11.9,1.0%,2017,1
3,11220,Coby Fleener,11.4,0.0%,2017,1
4,10003,Charles Clay,11.3,0.0%,2017,1
5,11689,Zach Ertz,9.3,59.2%,2017,1
6,9906,Kyle Rudolph,8.6,1.0%,2017,1
7,9172,Delanie Walker,7.6,0.1%,2017,1
8,11881,Demetrius Harris,7.5,0.0%,2017,1
9,9265,Jared Cook,5.6,0.0%,2017,1


In [33]:
# Output: a dataframe of weekly TE redzone stats week-by-week for defined years
def te_scrape_fantasypros_redzone_stats(start_year, end_year):
    """
    Scrapes FantasyPros TE Red Zone Stats for all weeks and years, including FantasyPros IDs.
    """
    week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in week_combinations:
        try:
            # Build the URL for TE
            url = f"https://www.fantasypros.com/nfl/red-zone-stats/te.php?year={year}&range=week&week={week}"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)
                df = pd.read_html(io.StringIO(table_str))[0]

                # Flatten multi-level headers if needed
                df.columns = df.columns.droplevel(0)
                df = df[df['Player'] != 'Player']  # Remove duplicate headers

                table_rows = table.find("tbody").find_all("tr")
                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"

                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        class_list = player_link.get("class", [])
                        for class_name in class_list:
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")
                                break
                        player_name = player_link.text.strip()

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names
                df["Year"] = year
                df["Week"] = week

                all_data.append(df)

            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    te_redzone_stats_df = pd.concat(all_data, ignore_index=True)

    columns_to_drop = ['Rank', 'ATT', 'YDS', 'TD', 'PCT', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST %']
    te_redzone_stats_df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

    # Rename columns to add 'rz' suffix
    te_redzone_stats_df.rename(
        columns={
            col: f"{col}_rz" for col in te_redzone_stats_df.columns
            if col not in ['Player', 'FantasyPros_ID', 'Year', 'Week']
        },
        inplace=True
    )

    # *** csv file ***
    # te_redzone_stats_df.to_csv("fantasypros_te_redzone_stats.csv", index=False)

    print(f"\n📊 **Shape of TE Red Zone Stats dataframe after processing:** {te_redzone_stats_df.shape}")
    display(te_redzone_stats_df.head(10))

    return te_redzone_stats_df

# ✅ Run the scraping function for all years
te_redzone_stats_df = te_scrape_fantasypros_redzone_stats(2017, years[-1])



📊 **Shape of TE Red Zone Stats dataframe after processing:** (3496, 9)


,FantasyPros_ID,Player,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,Year,Week
0,13959,Jesse James,2,3,66.7%,3.0,100.0%,2017,1
1,11220,Coby Fleener,3,3,100.0%,8.3,100.0%,2017,1
2,9906,Kyle Rudolph,2,2,100.0%,10.0,100.0%,2017,1
3,9275,Jason Witten,1,1,100.0%,12.0,100.0%,2017,1
4,11881,Demetrius Harris,1,1,100.0%,7.0,50.0%,2017,1
5,10003,Charles Clay,1,3,33.3%,1.0,100.0%,2017,1
6,14072,A.J. Derby,1,1,100.0%,13.0,100.0%,2017,1
7,9635,Zach Miller,1,2,50.0%,11.0,100.0%,2017,1
8,15623,Tyler Higbee,1,1,100.0%,9.0,100.0%,2017,1
9,16399,David Njoku,1,1,100.0%,7.0,100.0%,2017,1


In [34]:
# *** FantasyPros Dataframes from each section can be used as an alternative for merging ***

# Output: merged FantasyPros TE dataframe 
# Note: NOT finalized yet - needs data normalization 

# ✅ Display the shape of each dataframe before merging
print(f"\n📊 **Shape of TE Advanced Stats DataFrame:** {te_fp_advanced_stats_df.shape}")
print(f"📊 **Shape of TE FPTS and % Rostered DataFrame:** {te_fpts_perct_rost_df.shape}")
print(f"📊 **Shape of TE Red Zone Stats DataFrame:** {te_redzone_stats_df.shape}")

# ✅ Drop 'Player' from FPTS and redzone dataframes before merge
te_fpts_perct_rost_df = te_fpts_perct_rost_df.drop(columns=["Player"], errors="ignore")
te_redzone_stats_df = te_redzone_stats_df.drop(columns=["Player"], errors="ignore")

# ✅ Merge TE Advanced Stats with Fantasy Points and % Rostered
te_adv_fpts_rost_merged_df = pd.merge(
    te_fp_advanced_stats_df,  # Baseline DF (keeps 'Player')
    te_fpts_perct_rost_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'  # Retains all rows from the baseline DF
)

# ✅ Merge with Red Zone Stats
te_adv_fpts_rost_rz_merged_df = pd.merge(
    te_adv_fpts_rost_merged_df,
    te_redzone_stats_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'
)

# ✅ Sort for easier verification
te_adv_fpts_rost_rz_merged_df_sorted = te_adv_fpts_rost_rz_merged_df.sort_values(
    by=['Year', 'Week', 'YDS'], 
    ascending=[True, True, False]
)

# ✅ Display the updated shape after dropping duplicate columns
print(f"\n📊 **FantasyPros TE Shape After Merge:** {te_adv_fpts_rost_rz_merged_df_sorted.shape}")

# *** csv file ***
# te_adv_fpts_rost_rz_merged_df_sorted.to_csv('fantasypros_te_adv_fpts_rost_rz_df.csv', index=False)
display(te_adv_fpts_rost_rz_merged_df_sorted)


📊 **Shape of TE Advanced Stats DataFrame:** (1120, 22)
📊 **Shape of TE FPTS and % Rostered DataFrame:** (15142, 6)
📊 **Shape of TE Red Zone Stats DataFrame:** (3496, 9)

📊 **FantasyPros TE Shape After Merge:** (1120, 29)


,Year,Week,FantasyPros_ID,Player,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0,9.3,59.2%,NaN,NaN,NaN,NaN,NaN
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0,7.6,0.1%,NaN,NaN,NaN,NaN,NaN
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,11.9,1.0%,1.0,1.0,100.0%,12.0,100.0%
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,11.4,0.0%,3.0,3.0,100.0%,8.3,100.0%
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,11.3,0.0%,1.0,3.0,33.3%,1.0,100.0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,2024,18,17415,Jordan Akins,Jordan Akins (FA),1,6,60,36,66,24,8,0,8,6,0,2,2,1,0,0,0,12.0,1.0%,2.0,2.0,100.0%,9.0,100.0%
1116,2024,18,17269,Mark Andrews,Mark Andrews (BAL),1,4,54,54,126,0,1,0,8,5,1,1,2,1,1,0,0,11.4,93.6%,1.0,1.0,100.0%,12.0,100.0%
1117,2024,18,22955,Brock Bowers,Brock Bowers (LV),1,4,50,62,200,38,20,2,9,10,2,3,6,2,0,0,0,11.0,98.3%,1.0,3.0,33.3%,12.0,75.0%
1118,2024,18,15623,Tyler Higbee,Tyler Higbee (LAR),1,5,46,12,12,34,10,0,7,5,0,3,2,0,0,0,0,10.6,16.9%,1.0,3.0,33.3%,6.0,100.0%


In [35]:
### End:fantasypros webscraping ###

In [36]:
### Begin: FantasyPros Data Integrity Checks Section ###

In [37]:
## Data Integrity Checks ##
# Extract unique years and weeks from each dataframe
years_merged = sorted(te_adv_fpts_rost_rz_merged_df_sorted['Year'].unique())
weeks_merged = sorted(te_adv_fpts_rost_rz_merged_df_sorted['Week'].unique())

years_redzone = sorted(te_redzone_stats_df['Year'].unique())
weeks_redzone = sorted(te_redzone_stats_df['Week'].unique())

years_fpts = sorted(te_fpts_perct_rost_df['Year'].unique())
weeks_fpts = sorted(te_fpts_perct_rost_df['Week'].unique())

years_adv_stats = sorted(te_fp_advanced_stats_df['Year'].unique())
weeks_adv_stats = sorted(te_fp_advanced_stats_df['Week'].unique())

# Print the results for comparison
print(f"Years in Merged TE DF: {years_merged}")
print(f"Years in RedZone TE DF: {years_redzone}")
print(f"Years in FPTS TE DF: {years_fpts}")
print(f"Years in Advanced Stats TE DF: {years_adv_stats}\n")

print(f"Weeks in Merged TE DF: {weeks_merged}")
print(f"Weeks in RedZone TE DF: {weeks_redzone}")
print(f"Weeks in FPTS TE DF: {weeks_fpts}")
print(f"Weeks in Advanced Stats TE DF: {weeks_adv_stats}")


Years in Merged TE DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in RedZone TE DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in FPTS TE DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in Advanced Stats TE DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

Weeks in Merged TE DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in RedZone TE DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in FPTS TE DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in Advanced Stats TE DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [38]:
# ✅ Integrity Check for TE Advanced Stats (using FantasyPros_ID)
def test_te_advanced_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for TE advanced stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (REC, YDS, YBC, AIR, YAC, TGT, CATCHABLE).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("🔹 Running TE Advanced Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']\n")

    # Load data from memory or CSV
    merged_df = load_dataframe("te_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_te_adv_fpts_rost_rz_df.csv")
    adv_stats_df = load_dataframe("te_fp_advanced_stats_df", "fantasypros_te_advanced_stats.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(adv_stats_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        adv_year_df = adv_stats_df[adv_stats_df['Year'] == year]

        if merged_year_df.empty or adv_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                adv_row = adv_year_df[(adv_year_df['FantasyPros_ID'] == fp_id) & (adv_year_df['Week'] == week)]

                if merged_row.empty or adv_row.empty:
                    continue

                for col in ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    adv_val = adv_row[col].values[0] if col in adv_row else None

                    if merged_val is not None and adv_val is not None:
                        total_comparisons += 1
                        if merged_val == adv_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, adv_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_te_advanced_stats()


🔹 Running TE Advanced Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (630 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (924 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (658 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (819 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (672 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (812 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (602 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 602
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [39]:
def test_te_fpts_rost(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for TE Fantasy Points and % Rostered.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (FPTS, ROST).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("\n🔹 Running TE Fantasy Points & % Rostered Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['FPTS', 'ROST']\n")

    # Load data
    merged_df = load_dataframe("te_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_te_adv_fpts_rost_rz_df.csv")
    fpts_df = load_dataframe("te_fpts_perct_rost_df", "fantasypros_te_fpts_perct_rost.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(fpts_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        fpts_year_df = fpts_df[fpts_df['Year'] == year]

        if merged_year_df.empty or fpts_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                fpts_row = fpts_year_df[(fpts_year_df['FantasyPros_ID'] == fp_id) & (fpts_year_df['Week'] == week)]

                if merged_row.empty or fpts_row.empty:
                    continue

                for col in ['FPTS', 'ROST']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    fpts_val = fpts_row[col].values[0] if col in fpts_row else None

                    if merged_val is not None and fpts_val is not None:
                        total_comparisons += 1
                        if merged_val == fpts_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, fpts_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_te_fpts_rost()



🔹 Running TE Fantasy Points & % Rostered Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['FPTS', 'ROST']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (180 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (264 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (188 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (234 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (192 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (232 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (172 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 172
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [40]:
def test_te_redzone_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for TE Red Zone stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (REC_rz, TGT_rz, REC PCT_rz, Y/R_rz, TGT PCT_rz).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("\n🔹 Running TE Red Zone Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']\n")

    # Load data from memory or CSV
    merged_df = load_dataframe("te_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_te_adv_fpts_rost_rz_df.csv")
    redzone_df = load_dataframe("te_redzone_stats_df", "fantasypros_te_redzone_stats.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(redzone_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        redzone_year_df = redzone_df[redzone_df['Year'] == year]

        if merged_year_df.empty or redzone_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                redzone_row = redzone_year_df[(redzone_year_df['FantasyPros_ID'] == fp_id) & (redzone_year_df['Week'] == week)]

                if merged_row.empty or redzone_row.empty:
                    continue

                for col in ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    redzone_val = redzone_row[col].values[0] if col in redzone_row else None

                    if merged_val is not None and redzone_val is not None:
                        total_comparisons += 1
                        if merged_val == redzone_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, redzone_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_te_redzone_stats()



🔹 Running TE Red Zone Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (345 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (555 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (405 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (535 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (390 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (460 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (345 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 345
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [41]:
### End: FantasyPros Data Integrity Checks Section ###

In [42]:
## Begin: data normalization for fantasypros dataframes ##

In [43]:
# Pre-Normalization
# Output: display the columns to confirm all TE dataframes are present

# Establish the initial TE dataframes without any normalization
dataframes = {
    "Merged DataFrame": te_adv_fpts_rost_rz_merged_df_sorted,
    "Advanced Stats DataFrame": te_fp_advanced_stats_df,
    "FPTS & ROST DataFrame": te_fpts_perct_rost_df,
    "Redzone Stats DataFrame": te_redzone_stats_df
}

# Extract and compare column names
column_details = {name: df.columns.tolist() for name, df in dataframes.items()}

# Convert to DataFrame for easier comparison
columns_df = pd.DataFrame.from_dict(column_details, orient="index").transpose()

# Display column comparison
print("🔍 Initial Column Comparison Across TE DataFrames:\n")
print(columns_df)


🔍 Initial Column Comparison Across TE DataFrames:

   Merged DataFrame Advanced Stats DataFrame FPTS & ROST DataFrame Redzone Stats DataFrame
0              Year                     Year        FantasyPros_ID          FantasyPros_ID
1              Week                     Week                  FPTS                  REC_rz
2    FantasyPros_ID           FantasyPros_ID                  ROST                  TGT_rz
3            Player                   Player                  Year              REC PCT_rz
4            Player                   Player                  Week                  Y/R_rz
5                 G                        G                  None              TGT PCT_rz
6               REC                      REC                  None                    Year
7               YDS                      YDS                  None                    Week
8               YBC                      YBC                  None                    None
9               AIR                    

In [44]:
# Data Normalization - Rename Columns for Consistency
# Output: display columns

# Initialize TE dataframes and rename 'Year' to 'Season' where needed
dataframes = {
    "Merged DataFrame": te_adv_fpts_rost_rz_merged_df_sorted.copy(),
    "Advanced Stats DataFrame": te_fp_advanced_stats_df.copy(),
    "FPTS & ROST DataFrame": te_fpts_perct_rost_df.copy(),
    "Redzone Stats DataFrame": te_redzone_stats_df.copy()
}

# Rename 'Year' to 'Season' for consistency
for name, df in dataframes.items():
    if 'Year' in df.columns:
        df.rename(columns={'Year': 'Season'}, inplace=True)

# Check renamed columns
for name, df in dataframes.items():
    print(f"\n📊 {name} Columns After Renaming:")
    print(df.columns.tolist())


📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'FPTS', 'ROST', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS']

📊 FPTS & ROST DataFrame Columns After Renaming:
['FantasyPros_ID', 'FPTS', 'ROST', 'Season', 'Week']

📊 Redzone Stats DataFrame Columns After Renaming:
['FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'Season', 'Week']


In [45]:
# Data normalization - handle duplicate 'Player' columns in the merged and advanced stats dataframes
# Output: display columns in the merged and advanced stats dataframes

# Rename duplicate 'Player' columns for the relevant TE dataframes
def rename_player_columns(dataframes):
    # Rename 'Player' columns in the Merged DataFrame
    merged_df = dataframes["Merged DataFrame"]
    merged_df.columns.values[4] = "PlayerTM"  # Only renaming index 4

    # Rename 'Player' columns in the Advanced Stats DataFrame
    adv_stats_df = dataframes["Advanced Stats DataFrame"]
    adv_stats_df.columns.values[4] = "PlayerTM"  # Only renaming index 4

    # Confirm renaming
    print("\n📊 Merged DataFrame Columns After Renaming:")
    print(merged_df.columns.tolist())

    print("\n📊 Advanced Stats DataFrame Columns After Renaming:")
    print(adv_stats_df.columns.tolist())

# Apply the renaming function to the TE dataframes
rename_player_columns(dataframes)


📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'FPTS', 'ROST', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS']


In [46]:
# Data normalization - confirm renamed 'Player' column in the merged and advanced stats dataframes
# Output: display first five rows in the merged and advanced stats dataframes

# Display the first 5 rows of the Merged DataFrame
print("\n🔍 First 5 Rows of TE Merged DataFrame:")
display(dataframes["Merged DataFrame"].head())

# Display the first 5 rows of the Advanced Stats DataFrame
print("\n🔍 First 5 Rows of TE Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"].head())


🔍 First 5 Rows of TE Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0,9.3,59.2%,NaN,NaN,NaN,NaN,NaN
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0,7.6,0.1%,NaN,NaN,NaN,NaN,NaN
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,11.9,1.0%,1.0,1.0,100.0%,12.0,100.0%
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,11.4,0.0%,3.0,3.0,100.0%,8.3,100.0%
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,11.3,0.0%,1.0,3.0,33.3%,1.0,100.0%



🔍 First 5 Rows of TE Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS
0,2017,1,15581,Austin Hooper,Austin Hooper (NE),1,2,128,45,0,83,65,3,2,2,0,0,2,2,2,2,1
1,2017,1,13959,Jesse James,Jesse James (FA),1,6,41,15,0,26,7,0,8,6,0,2,1,0,0,0,0
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0


In [47]:
# Data normalization - reorder columns
# Output: first three columns for all dataframes: 'Season', 'Week', 'FantasyPros_ID'

# Define a helper function to reorder columns in-place
def reorder_columns_inplace(df, first_cols=['Season', 'Week', 'FantasyPros_ID']):
    """Reorders the DataFrame columns to place first_cols at the beginning in-place."""
    current_cols = list(df.columns)
    remaining_cols = [col for col in current_cols if col not in first_cols]
    reordered_df = df[first_cols + remaining_cols]
    return reordered_df

# Apply the function to each TE dataframe separately
dataframes["Merged DataFrame"] = reorder_columns_inplace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = reorder_columns_inplace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = reorder_columns_inplace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = reorder_columns_inplace(dataframes["Redzone Stats DataFrame"])

# Confirm order
for name, df in dataframes.items():
    print(f"\n🔍 {name} First Three Columns: {df.columns[:3].tolist()}")


🔍 Merged DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Advanced Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 FPTS & ROST DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Redzone Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']


In [48]:
# Pre-Normalization - Initial Data Type Inspection
# Output: columns and data types

# Inspect data types of each TE dataframe before applying normalization
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season              int64
Week                int64
FantasyPros_ID     object
Player             object
PlayerTM           object
G                  object
REC                object
YDS                object
YBC                object
AIR                object
YAC                object
YACON              object
BRKTKL             object
TGT                object
CATCHABLE          object
DROP               object
RZ TGT             object
10+ YDS            object
20+ YDS            object
30+ YDS            object
40+ YDS            object
50+ YDS            object
FPTS              float64
ROST               object
REC_rz            float64
TGT_rz            float64
REC PCT_rz         object
Y/R_rz            float64
TGT PCT_rz         object
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season             int64
Week               int64
FantasyPros_ID    object
Player            object
PlayerTM          object
G          

In [49]:
# data normalization - convert all columns to object (str) types
# no output

# Helper Function: Convert to Object & Strip Whitespaces
def convert_to_object_and_strip(df):
    """
    Converts all columns in a DataFrame to object type and strips leading/trailing whitespaces.
    """
    df_copy = df.copy()  # Create a copy to avoid modifying the original dataframe

    for col in df_copy.columns:
        # Convert to object type
        df_copy[col] = df_copy[col].astype(str).apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    return df_copy

In [50]:
# Apply the function to all TE dataframes
dataframes = {name: convert_to_object_and_strip(df) for name, df in dataframes.items()}

# Confirm data type conversion
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season            object
Week              object
FantasyPros_ID    object
Player            object
PlayerTM          object
G                 object
REC               object
YDS               object
YBC               object
AIR               object
YAC               object
YACON             object
BRKTKL            object
TGT               object
CATCHABLE         object
DROP              object
RZ TGT            object
10+ YDS           object
20+ YDS           object
30+ YDS           object
40+ YDS           object
50+ YDS           object
FPTS              object
ROST              object
REC_rz            object
TGT_rz            object
REC PCT_rz        object
Y/R_rz            object
TGT PCT_rz        object
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season            object
Week              object
FantasyPros_ID    object
Player            object
PlayerTM          object
G                 object
REC            

In [51]:
# data normalization - verification of data types
# output: a display of all columns in all dataframes as 'object' datatypes 

def compare_column_dtypes(dataframes, merged_df_name="Merged DataFrame"):
    """
    Compare column data types between the merged dataframe and each unmerged dataframe.

    Parameters:
    - dataframes: dict of dataframes to compare.
    - merged_df_name: Name of the merged dataframe in the dictionary.

    Returns:
    - Comparison results printed to the console.
    """
    merged_df = dataframes[merged_df_name]
    merged_dtypes = merged_df.dtypes.to_dict()

    comparison_results = {}

    for name, df in dataframes.items():
        if name == merged_df_name:
            continue  # Skip comparison with itself

        df_dtypes = df.dtypes.to_dict()
        mismatch_report = []

        for col, dtype in df_dtypes.items():
            if col in merged_dtypes:
                if dtype != merged_dtypes[col]:
                    mismatch_report.append((col, dtype, merged_dtypes[col]))

        if mismatch_report:
            comparison_results[name] = mismatch_report
        else:
            comparison_results[name] = "✅ All data types match"

    # Display results
    print("\n🔍 Column Data Type Comparison Results:\n")
    for name, result in comparison_results.items():
        if result == "✅ All data types match":
            print(f"{name}: ✅ All data types match")
        else:
            print(f"\n{name}: ❌ Mismatched Columns Detected")
            for col, dtype_unmerged, dtype_merged in result:
                print(f"  - Column: {col} | Unmerged Type: {dtype_unmerged} | Merged Type: {dtype_merged}")

# Run the comparison
compare_column_dtypes(dataframes)


🔍 Column Data Type Comparison Results:

Advanced Stats DataFrame: ✅ All data types match
FPTS & ROST DataFrame: ✅ All data types match
Redzone Stats DataFrame: ✅ All data types match


In [52]:
# data normalization - verification of column order
# output: display of the first three columns of all dataframes

# Define the correct order of columns
expected_first_columns = ['Season', 'Week', 'FantasyPros_ID']

# Function to check column order
def check_column_order(dataframes, expected_first_columns):
    """
    Check if the first columns of each dataframe match the expected columns.

    Parameters:
    - dataframes: dict of dataframes to check.
    - expected_first_columns: list of column names expected to appear first.

    Returns:
    - Prints the order check results for each dataframe.
    """
    print("\n🔍 Column Order Verification:\n")
    for name, df in dataframes.items():
        first_columns = df.columns[:len(expected_first_columns)].tolist()
        
        if first_columns == expected_first_columns:
            print(f"{name}: ✅ Columns are in the correct order: {first_columns}")
        else:
            print(f"{name}: ❌ Incorrect column order. Found: {first_columns} | Expected: {expected_first_columns}")

# Run the column order check
check_column_order(dataframes, expected_first_columns)


🔍 Column Order Verification:

Merged DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Advanced Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
FPTS & ROST DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Redzone Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']


In [53]:
# data normalization - check to ensure there are no missing columns
# output: an empty dictionary {}

# Define expected columns for each dataframe
expected_columns = {
    "Merged DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 
        'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', 
        '30+ YDS', '40+ YDS', '50+ YDS', 'FPTS', 'ROST', 'REC_rz', 'TGT_rz', 
        'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz'
    ],
    "Advanced Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 
        'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', 
        '30+ YDS', '40+ YDS', '50+ YDS'
    ],
    "FPTS & ROST DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST'
    ],
    "Redzone Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 
        'Y/R_rz', 'TGT PCT_rz'
    ]
}

# Check for missing and unexpected columns in each dataframe
column_issues = {}

for df_name, df in dataframes.items():
    expected = set(expected_columns[df_name])
    actual = set(df.columns.tolist())
    
    missing = expected - actual
    unexpected = actual - expected
    
    if missing or unexpected:
        column_issues[df_name] = {
            "Missing Columns": list(missing) if missing else None,
            "Unexpected Columns": list(unexpected) if unexpected else None
        }

# Display results
if column_issues:
    print("🔍 Column Presence Issues Detected:\n")
    for df_name, issues in column_issues.items():
        print(f"{df_name}:")
        if issues["Missing Columns"]:
            print(f"  ❌ Missing Columns: {issues['Missing Columns']}")
        if issues["Unexpected Columns"]:
            print(f"  ⚠️  Unexpected Columns: {issues['Unexpected Columns']}")
else:
    print("✅ All columns are present and consistent across all dataframes.")

✅ All columns are present and consistent across all dataframes.


In [54]:
# data normalization - remove leading and trailing whitespaces from all object (string) columns 
# no output

def remove_whitespace(df, inplace=False):
    if not inplace:
        df = df.copy()  # Only create a copy if inplace is False

    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()  # Automatically skips non-string values
    
    if not inplace:
        return df  # Return the modified DataFrame if not modifying in place


In [55]:
# data normalization - remove white spaces
# output: dataframes stripped of whitespaces

# Apply whitespace removal to each TE dataframe separately
dataframes["Merged DataFrame"] = remove_whitespace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = remove_whitespace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = remove_whitespace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = remove_whitespace(dataframes["Redzone Stats DataFrame"])

# Confirm whitespace removal by displaying first few rows
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Whitespace Removal):")
    print(df.head(5))


🔍 Merged DataFrame - Sample Data (Post Whitespace Removal):
  Season Week FantasyPros_ID          Player             PlayerTM  G REC YDS YBC AIR YAC YACON BRKTKL TGT CATCHABLE DROP RZ TGT 10+ YDS 20+ YDS 30+ YDS 40+ YDS 50+ YDS  FPTS   ROST REC_rz TGT_rz REC PCT_rz Y/R_rz TGT PCT_rz
5   2017    1          11689       Zach Ertz      Zach Ertz (WAS)  1   8  93  78   0  15     5      0   8         8    0      0       5       1       0       0       0   9.3  59.2%    nan    nan        nan    nan        nan
7   2017    1           9172  Delanie Walker  Delanie Walker (FA)  1   7  76  67   0   9     4      0   9         7    0      0       4       1       0       0       0   7.6   0.1%    nan    nan        nan    nan        nan
2   2017    1           9275    Jason Witten    Jason Witten (FA)  1   7  59  47   0  12     4      0   9         7    0      1       3       0       0       0       0  11.9   1.0%    1.0    1.0     100.0%   12.0     100.0%
3   2017    1          11220    Coby Fleene

In [56]:
# data normalization  - standardize decimal places for numerical data and percentages
# no output
def format_precision(df, precision_map=None):
    if precision_map is None:
        precision_map = {
            'REC PCT_rz': '.2f',
            'TGT PCT_rz': '.2f',
            'ROST': '.3f',
            'Y/R_rz': '.1f'
        }

    df_copy = df.copy()
    
    for col, fmt in precision_map.items():
        if col in df_copy.columns:
            try:
                df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
                df_copy[col] = df_copy[col].map(lambda x: format(x, fmt) if pd.notna(x) else "nan")
            except Exception as e:
                print(f"⚠️ Error formatting column {col}: {e}")
    
    return df_copy

In [57]:
# data normalization - apply decimal place precision  
# output

# Define precision map (same for all relevant dataframes)
precision_map = {
    'REC PCT_rz': '.2f',
    'TGT PCT_rz': '.2f',
    'ROST': '.3f',
    'Y/R_rz': '.1f'
}

# Apply formatting function to each dataframe
dataframes["Merged DataFrame"] = format_precision(dataframes["Merged DataFrame"], precision_map)
dataframes["Advanced Stats DataFrame"] = format_precision(dataframes["Advanced Stats DataFrame"], precision_map)
dataframes["FPTS & ROST DataFrame"] = format_precision(dataframes["FPTS & ROST DataFrame"], precision_map)
dataframes["Redzone Stats DataFrame"] = format_precision(dataframes["Redzone Stats DataFrame"], precision_map)

# Confirm precision formatting by displaying relevant columns
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Precision Formatting):")
    display(df.head(5))


🔍 Merged DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0,9.3,nan,nan,nan,nan,nan,nan
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0,7.6,nan,nan,nan,nan,nan,nan
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,11.9,nan,1.0,1.0,nan,12.0,nan
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,11.4,nan,3.0,3.0,nan,8.3,nan
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,11.3,nan,1.0,3.0,nan,1.0,nan



🔍 Advanced Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS
0,2017,1,15581,Austin Hooper,Austin Hooper (NE),1,2,128,45,0,83,65,3,2,2,0,0,2,2,2,2,1
1,2017,1,13959,Jesse James,Jesse James (FA),1,6,41,15,0,26,7,0,8,6,0,2,1,0,0,0,0
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0



🔍 FPTS & ROST DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,FPTS,ROST
0,2017,1,15581,18.8,nan
1,2017,1,13959,16.1,nan
2,2017,1,9275,11.9,nan
3,2017,1,11220,11.4,nan
4,2017,1,10003,11.3,nan



🔍 Redzone Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
0,2017,1,13959,2,3,nan,3.0,nan
1,2017,1,11220,3,3,nan,8.3,nan
2,2017,1,9906,2,2,nan,10.0,nan
3,2017,1,9275,1,1,nan,12.0,nan
4,2017,1,11881,1,1,nan,7.0,nan


In [58]:
## End: data normalization for fantasypros dataframes ##

In [59]:
### Begin: Enhanced data integrity checks for the fantasypros dataframes ###

In [60]:
# Enhanced data integrity checks
# no output
def generate_md5_hash(row):
    """
    Generate a consistent MD5 hash string from a row's values.
    
    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    
    Returns:
    - Hash string (MD5)
    """
    row_str = "|".join(map(str, row))
    return hashlib.md5(row_str.encode()).hexdigest()

In [61]:
# Enhanced data integrity checks
# output: Merged DataFrame with a hash column

# Generate hashes for the Merged DataFrame
dataframes["Merged DataFrame"]["row_hash"] = dataframes["Merged DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Merged DataFrame - Hash Generation Complete")
display(dataframes["Merged DataFrame"].head(5))


✅ Merged DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0,9.3,nan,nan,nan,nan,nan,nan,ef1b8353cf1d16129ecfd26737af0f60
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0,7.6,nan,nan,nan,nan,nan,nan,798092a5f2aff613c27574bdfacf2f26
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,11.9,nan,1.0,1.0,nan,12.0,nan,c7d2e6043b9773164070c1236677d7c7
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,11.4,nan,3.0,3.0,nan,8.3,nan,5e14f8142be9ee32702c892763c7ed04
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,11.3,nan,1.0,3.0,nan,1.0,nan,79e791de4b1b7339a26de87e94ff415e


In [62]:
# Enhanced data integrity checks
# output: Advanced Stat DataFrame with a hash column

# Generate hashes for the Advanced Stats DataFrame
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Advanced Stats DataFrame - Hash Generation Complete")
display(dataframes["Advanced Stats DataFrame"].head(5))



✅ Advanced Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,row_hash
0,2017,1,15581,Austin Hooper,Austin Hooper (NE),1,2,128,45,0,83,65,3,2,2,0,0,2,2,2,2,1,ef4a176a91b60e04805b0f7010878416
1,2017,1,13959,Jesse James,Jesse James (FA),1,6,41,15,0,26,7,0,8,6,0,2,1,0,0,0,0,ba5a689d2126458c00e13d3f4bce1e47
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,348d53951361d9ed06bbe00def463907
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,2d952cc0fc488dc4647aa4caff1d7ff6
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,a72857d011e2a7e33a0d1b9d4b4da3a2


In [63]:
# Enhanced data integrity checks
# output: FPTS & ROST DataFrame with a hash column

# Generate hashes for the FPTS & ROST DataFrame
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ FPTS & ROST DataFrame - Hash Generation Complete")
display(dataframes["FPTS & ROST DataFrame"].head(5))



✅ FPTS & ROST DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,FPTS,ROST,row_hash
0,2017,1,15581,18.8,nan,c565fa6d1a2d4d223ad8f4c9fc14ebc1
1,2017,1,13959,16.1,nan,b23eaf7729da1d4cac35e606aa2f0368
2,2017,1,9275,11.9,nan,d6c9ca727ee0317d18fca49f80297627
3,2017,1,11220,11.4,nan,7ebc5a1760c9746b447dc94a2eb48f1c
4,2017,1,10003,11.3,nan,3750c2788501e9f6761bd08ec818a946


In [64]:
# Enhanced data integrity checks
# output: Redzone Stats DataFrame with a hash column

# Generate hashes for the Redzone Stats DataFrame
dataframes["Redzone Stats DataFrame"]["row_hash"] = dataframes["Redzone Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Redzone Stats DataFrame - Hash Generation Complete")
display(dataframes["Redzone Stats DataFrame"].head(5))


✅ Redzone Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash
0,2017,1,13959,2,3,nan,3.0,nan,8376e1036e905695439200626d960298
1,2017,1,11220,3,3,nan,8.3,nan,bbd75552d5bca621df88a72c43e0a510
2,2017,1,9906,2,2,nan,10.0,nan,68181af8b8f7ed9701508f575a9f8e90
3,2017,1,9275,1,1,nan,12.0,nan,1d2596b8aeeed31cac5a0607c1aabd0c
4,2017,1,11881,1,1,nan,7.0,nan,baa87afe055d5063105cde763bda0664


In [65]:
# Enhanced data integrity checks
# output: renamed the merged dataframe 

# Create a copy of the merged dataframe with "_hash" suffix for modification
te_adv_fpts_rost_rz_merged_df_sorted_hash = dataframes["Merged DataFrame"].copy()

# Verify the creation of the new dataframe
print("\n✅ New Merged DataFrame Created: 'te_adv_fpts_rost_rz_merged_df_sorted_hash'")


✅ New Merged DataFrame Created: 'te_adv_fpts_rost_rz_merged_df_sorted_hash'


In [66]:
# Enhanced data integrity checks
# output: columns of merged dataframe with new hash columns

# Define the new columns to be added for hashing
new_hash_columns = ['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']

# Initialize the new columns with empty strings (or NaN if preferred)
for col in new_hash_columns:
    te_adv_fpts_rost_rz_merged_df_sorted_hash[col] = ""  # Empty strings as placeholders

# Verify creation of columns
print("\n✅ Hash columns added to the new merged dataframe:")
print(te_adv_fpts_rost_rz_merged_df_sorted_hash.columns.tolist())


✅ Hash columns added to the new merged dataframe:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'FPTS', 'ROST', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']


In [67]:
# Enhanced data integrity checks
# output: last 5 columns of the new merged dataframe with row hashes

# Display the last 5 columns to confirm the new hash columns are present
print("\n🔍 Last 5 Columns in the New Merged DataFrame:")
print(te_adv_fpts_rost_rz_merged_df_sorted_hash.columns[-5:])


🔍 Last 5 Columns in the New Merged DataFrame:
Index(['TGT PCT_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash'], dtype='object')


In [68]:
# Enhanced data integrity checks - define Relevant Columns for Each Hash Type
# output: columns to be hashed

# Define the columns to be used for each hash type
hash_column_groups = {
    "adv_stats_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 
        'TGT', 'CATCHABLE', 'DROP', 'RZ TGT'
    ],
    "fpts_rost_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST'
    ],
    "redzone_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz'
    ]
}

# Confirm the column groups are set correctly
print("\n✅ Hash Column Groups Defined:")
for hash_name, columns in hash_column_groups.items():
    print(f"  - {hash_name}: {columns}")


✅ Hash Column Groups Defined:
  - adv_stats_hash: ['Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT']
  - fpts_rost_hash: ['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']
  - redzone_hash: ['Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']


In [69]:
# Enhanced data integrity checks - view a sample of the hashes
# output: preview of row hashes in the merged dataframe

# Apply hashing for each hash type using the existing generate_md5_hash function
for hash_name, columns in hash_column_groups.items():
    te_adv_fpts_rost_rz_merged_df_sorted_hash[hash_name] = te_adv_fpts_rost_rz_merged_df_sorted_hash[columns].apply(generate_md5_hash, axis=1)

# Confirm hashes were generated
print("\n✅ Hashes Generated for All Hash Types.")

# Display the first 5 rows of the hash columns for verification
display(te_adv_fpts_rost_rz_merged_df_sorted_hash[['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']].head())


✅ Hashes Generated for All Hash Types.


,adv_stats_hash,fpts_rost_hash,redzone_hash
5,ddd26d3136e455f0bac718ecac98485b,b36ba8a3b8cbf6a0972d07aa073ad231,142e6ca07530d5d83b8f4e37cbcaae7e
7,598bdec682c30cf8fb22c8829beba718,1b311e6e33fcaa7a207d3aa4fe4dbfc5,3901cff380246e48a1cccef1819e7450
2,1a76e826c5884f9a916c24319831c3dc,d6c9ca727ee0317d18fca49f80297627,4dd346b5a333b36517994fcabc710f7d
3,c98e21a749ab69f9300587c9edb4601c,7ebc5a1760c9746b447dc94a2eb48f1c,9ea1f767e4a3d62e18cad59ce2e20ffd
4,280a8d2aa334cabf93e54c543d0a9028,3750c2788501e9f6761bd08ec818a946,9e39e7924ac6daeb08d086b0f2830c39


In [70]:
# ** Final fantasypros merged dataframe **

# Enhanced data integrity checks - view a sample of the hashes
# output: preview of row hashes in the merged dataframe

# Display the first 5 rows of the merged dataframe for verification
print("\n🔍 First 5 Rows of the TE Merged DataFrame:")
display(te_adv_fpts_rost_rz_merged_df_sorted_hash.head(5))


🔍 First 5 Rows of the TE Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash,adv_stats_hash,fpts_rost_hash,redzone_hash
5,2017,1,11689,Zach Ertz,Zach Ertz (WAS),1,8,93,78,0,15,5,0,8,8,0,0,5,1,0,0,0,9.3,nan,nan,nan,nan,nan,nan,ef1b8353cf1d16129ecfd26737af0f60,ddd26d3136e455f0bac718ecac98485b,b36ba8a3b8cbf6a0972d07aa073ad231,142e6ca07530d5d83b8f4e37cbcaae7e
7,2017,1,9172,Delanie Walker,Delanie Walker (FA),1,7,76,67,0,9,4,0,9,7,0,0,4,1,0,0,0,7.6,nan,nan,nan,nan,nan,nan,798092a5f2aff613c27574bdfacf2f26,598bdec682c30cf8fb22c8829beba718,1b311e6e33fcaa7a207d3aa4fe4dbfc5,3901cff380246e48a1cccef1819e7450
2,2017,1,9275,Jason Witten,Jason Witten (FA),1,7,59,47,0,12,4,0,9,7,0,1,3,0,0,0,0,11.9,nan,1.0,1.0,nan,12.0,nan,c7d2e6043b9773164070c1236677d7c7,1a76e826c5884f9a916c24319831c3dc,d6c9ca727ee0317d18fca49f80297627,4dd346b5a333b36517994fcabc710f7d
3,2017,1,11220,Coby Fleener,Coby Fleener (FA),1,5,54,42,0,12,9,0,6,6,0,2,2,1,0,0,0,11.4,nan,3.0,3.0,nan,8.3,nan,5e14f8142be9ee32702c892763c7ed04,c98e21a749ab69f9300587c9edb4601c,7ebc5a1760c9746b447dc94a2eb48f1c,9ea1f767e4a3d62e18cad59ce2e20ffd
4,2017,1,10003,Charles Clay,Charles Clay (FA),1,4,53,29,0,24,4,0,9,8,1,3,2,1,1,0,0,11.3,nan,1.0,3.0,nan,1.0,nan,79e791de4b1b7339a26de87e94ff415e,280a8d2aa334cabf93e54c543d0a9028,3750c2788501e9f6761bd08ec818a946,9e39e7924ac6daeb08d086b0f2830c39


In [71]:
# Enhanced data integrity checks - Generate Row Hashes for Advanced Stats DataFrame
# output: row hashes for the Advanced Stats DataFrame

# Define columns used for generating 'adv_stats_hash'
adv_stats_columns = [
    'Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 
    'TGT', 'CATCHABLE', 'DROP', 'RZ TGT'
]

# Ensure all columns are present in the dataframe
adv_stats_columns = [col for col in adv_stats_columns if col in dataframes["Advanced Stats DataFrame"].columns]

# Generate hashes for the Advanced Stats DataFrame
print("\n📊 Generating Hashes for: Advanced Stats DataFrame")
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"][adv_stats_columns].apply(generate_md5_hash, axis=1)

# Display a preview to confirm hashes were generated
print("Preview of Generated Hashes in Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())


📊 Generating Hashes for: Advanced Stats DataFrame
Preview of Generated Hashes in Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,15581,1f2809c85b9d43f14e5653b0f9ebe9ab
1,2017,1,13959,8377e0c1be6ea4af041c5750ef3dd8a1
2,2017,1,9275,1a76e826c5884f9a916c24319831c3dc
3,2017,1,11220,c98e21a749ab69f9300587c9edb4601c
4,2017,1,10003,280a8d2aa334cabf93e54c543d0a9028


In [72]:
# Enhanced data integrity checks - Prepare DataFrames for Comparison
# output: no output

# Select relevant columns from the Advanced Stats DataFrame
adv_stats_subset = dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']]

# Prepare the Merged DataFrame subset for comparison
merged_subset = te_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'adv_stats_hash']]

In [73]:
# Enhanced data integrity checks - Create a merge dataFrame for hash comparison
# output: a merged dataframe
# output: df contains the advanced stats column (merged df) and the row hash (advanced stats df) 

# Merge the dataframes on the keys - JOIN ON FantasyPros_ID
comparison_df = pd.merge(
    merged_subset,
    adv_stats_subset,
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_original')
)

# Display the merged comparison dataframe
print("Comparison: adv_stats_hash column (te_adv_fpts_rost_rz_merged_df_sorted_hash)")
print("Comparison: row_hash column (Advanced Stats DataFrame)\n")
display(comparison_df.head())


Comparison: adv_stats_hash column (te_adv_fpts_rost_rz_merged_df_sorted_hash)
Comparison: row_hash column (Advanced Stats DataFrame)



,Season,Week,FantasyPros_ID,adv_stats_hash,row_hash
0,2017,1,11689,ddd26d3136e455f0bac718ecac98485b,ddd26d3136e455f0bac718ecac98485b
1,2017,1,9172,598bdec682c30cf8fb22c8829beba718,598bdec682c30cf8fb22c8829beba718
2,2017,1,9275,1a76e826c5884f9a916c24319831c3dc,1a76e826c5884f9a916c24319831c3dc
3,2017,1,11220,c98e21a749ab69f9300587c9edb4601c,c98e21a749ab69f9300587c9edb4601c
4,2017,1,10003,280a8d2aa334cabf93e54c543d0a9028,280a8d2aa334cabf93e54c543d0a9028


In [74]:
# Enhanced data integrity checks - Identify Matches and Mismatches
# output: hopefully no mismatches

# Identify mismatches where hashes don't align
mismatches = comparison_df[comparison_df['adv_stats_hash'] != comparison_df['row_hash']]

# Display mismatch results
print(f"\n❌ Mismatches Found: {len(mismatches)}")
if len(mismatches) > 0:
    display(mismatches.head())
else:
    print("✅ All hashes match.")


❌ Mismatches Found: 0
✅ All hashes match.


In [75]:
# Enhanced data integrity checks - Generate Row Hashes for FPTS & ROST DataFrame
# output: Row hashes for FPTS & ROST DataFrame

# Display current dataframe being processed
print("\n📊 Processing: FPTS & ROST DataFrame")

# Generate row hashes for FPTS & ROST DataFrame using the existing helper function
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']].apply(generate_md5_hash, axis=1)

# Display a preview of the generated hashes
display(dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())


📊 Processing: FPTS & ROST DataFrame


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,15581,c565fa6d1a2d4d223ad8f4c9fc14ebc1
1,2017,1,13959,b23eaf7729da1d4cac35e606aa2f0368
2,2017,1,9275,d6c9ca727ee0317d18fca49f80297627
3,2017,1,11220,7ebc5a1760c9746b447dc94a2eb48f1c
4,2017,1,10003,3750c2788501e9f6761bd08ec818a946


In [76]:
# Enhanced data integrity checks - Check Matching IDs Before Comparison
# output: 

# Checking Matching IDs Between the DataFrames
print("\n📊 Checking Matching IDs Before Comparison:")

# Extract IDs from both dataframes
ids_unmerged = dataframes["FPTS & ROST DataFrame"]["FantasyPros_ID"].unique()
ids_merged = te_adv_fpts_rost_rz_merged_df_sorted_hash["FantasyPros_ID"].unique()

# Find intersection of IDs
matching_ids = set(ids_unmerged).intersection(set(ids_merged))
print(f"🔍 Total Matching IDs Found: {len(matching_ids)}")


📊 Checking Matching IDs Before Comparison:
🔍 Total Matching IDs Found: 161


In [77]:
# Enhanced data integrity checks - Compare Hashes 
# output: Hopefully no mismatches

# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df = pd.merge(
    te_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'fpts_rost_hash']],
    dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches = comparison_df[comparison_df['fpts_rost_hash'] != comparison_df['row_hash']]

# Display comparison results
if mismatches.empty:
    print("\n✅ All hashes match for FPTS & ROST DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches)}")
    display(mismatches.head())

# Preview of matched hashes
display(comparison_df.head())


✅ All hashes match for FPTS & ROST DataFrame.


,Season,Week,FantasyPros_ID,fpts_rost_hash,row_hash
0,2017,1,11689,b36ba8a3b8cbf6a0972d07aa073ad231,b36ba8a3b8cbf6a0972d07aa073ad231
1,2017,1,9172,1b311e6e33fcaa7a207d3aa4fe4dbfc5,1b311e6e33fcaa7a207d3aa4fe4dbfc5
2,2017,1,9275,d6c9ca727ee0317d18fca49f80297627,d6c9ca727ee0317d18fca49f80297627
3,2017,1,11220,7ebc5a1760c9746b447dc94a2eb48f1c,7ebc5a1760c9746b447dc94a2eb48f1c
4,2017,1,10003,3750c2788501e9f6761bd08ec818a946,3750c2788501e9f6761bd08ec818a946


In [78]:
## Fixme: **No row hash validation for the redzone data**  
# Enhanced data integrity checks - Compare Hashes for Redzone Data
# output: Display mismatches if found

# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df_redzone = pd.merge(
    te_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'redzone_hash']],
    dataframes["Redzone Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches_redzone = comparison_df_redzone[comparison_df_redzone['redzone_hash'] != comparison_df_redzone['row_hash']]

# Display comparison results
if mismatches_redzone.empty:
    print("\n✅ All hashes match for Redzone Stats DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches_redzone)}")
    display(mismatches_redzone.head())

# Display the first few rows of the comparison dataframe for reference
display(comparison_df_redzone.head())


❌ Mismatches Found: 920


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,9275,4dd346b5a333b36517994fcabc710f7d,1d2596b8aeeed31cac5a0607c1aabd0c
1,2017,1,11220,9ea1f767e4a3d62e18cad59ce2e20ffd,bbd75552d5bca621df88a72c43e0a510
2,2017,1,10003,9e39e7924ac6daeb08d086b0f2830c39,dd5d023ddd90259136ca138a12482c6c
3,2017,1,13959,520ee0517a0b955ed27434d6903480b3,8376e1036e905695439200626d960298
4,2017,1,9906,d453db0132a432497153390180678537,68181af8b8f7ed9701508f575a9f8e90


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,9275,4dd346b5a333b36517994fcabc710f7d,1d2596b8aeeed31cac5a0607c1aabd0c
1,2017,1,11220,9ea1f767e4a3d62e18cad59ce2e20ffd,bbd75552d5bca621df88a72c43e0a510
2,2017,1,10003,9e39e7924ac6daeb08d086b0f2830c39,dd5d023ddd90259136ca138a12482c6c
3,2017,1,13959,520ee0517a0b955ed27434d6903480b3,8376e1036e905695439200626d960298
4,2017,1,9906,d453db0132a432497153390180678537,68181af8b8f7ed9701508f575a9f8e90


In [79]:
### End: Enhanced data integrity checks for the fantasypros dataframes ##

In [80]:
## Begin: webscraping sportsoddshistory betting lines data ###

In [81]:
# Generate Dynamic URLs for the regular season 
# output: urls with each year to scrape

# Helper: Define year range
def get_year_range(current_year, current_week):
    return list(range(2017, current_year + 1)) if current_week >= 1 else list(range(2017, current_year))

# Current time context
current_year = datetime.now().year
current_week = datetime.now().isocalendar()[1]

# Get the range and build the season URLs
season_list = get_year_range(current_year, current_week)
season_urls = [f"https://www.sportsoddshistory.com/nfl-game-season/?y={year}" for year in season_list]

# Preview
for season, url in zip(season_list, season_urls):
    print(f"{season}: {url}")

2017: https://www.sportsoddshistory.com/nfl-game-season/?y=2017
2018: https://www.sportsoddshistory.com/nfl-game-season/?y=2018
2019: https://www.sportsoddshistory.com/nfl-game-season/?y=2019
2020: https://www.sportsoddshistory.com/nfl-game-season/?y=2020
2021: https://www.sportsoddshistory.com/nfl-game-season/?y=2021
2022: https://www.sportsoddshistory.com/nfl-game-season/?y=2022
2023: https://www.sportsoddshistory.com/nfl-game-season/?y=2023
2024: https://www.sportsoddshistory.com/nfl-game-season/?y=2024
2025: https://www.sportsoddshistory.com/nfl-game-season/?y=2025


In [82]:
# Load Soup and Extract Regular Season Week Headers
# output: regular season years

# Container for (season, soup, regular season week headers)
tm_game_odds_season_soup_headers = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Fetching {season}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all <h3> tags
    all_headers = soup.find_all('h3')

    # Only keep regular season week headers (e.g., "2023 Regular Season - Week 1")
    reg_week_headers = [tag for tag in all_headers if f"{season} Regular Season - Week" in tag.text]

    # Store tuple for next stage
    tm_game_odds_season_soup_headers.append((season, soup, reg_week_headers))

🔄 Fetching 2017...
🔄 Fetching 2018...
🔄 Fetching 2019...
🔄 Fetching 2020...
🔄 Fetching 2021...
🔄 Fetching 2022...
🔄 Fetching 2023...
🔄 Fetching 2024...
🔄 Fetching 2025...


In [83]:
# Scrape All Weeks for Each Year
# output: none

# Container to collect team-level weekly DataFrames
tm_game_odds_regular_team_level_list = []

for season, soup, week_headers in tm_game_odds_season_soup_headers:
    for tag in week_headers:
        week = int(tag.text.strip().split("Week")[1])

        table = tag.find_next('table')
        if not table:
            continue
        rows = table.find('tbody').find_all('tr')

        base_data = []
        abbr_data = []

        for row in rows:
            cells = row.find_all('td')
            if len(cells) != 11:
                continue

            base_data.append([cell.get_text(strip=True) for cell in cells])

            fav_link = cells[4].find('a')
            und_link = cells[8].find('a')

            fav_abbr = fav_link['href'].split('=')[1].split('&')[0] if fav_link else None
            und_abbr = und_link['href'].split('=')[1].split('&')[0] if und_link else None

            abbr_data.append([fav_abbr, und_abbr])

        if not base_data:
            continue

        df = pd.DataFrame(base_data, columns=[
            'Day', 'Date', 'Time (ET)', 'Home_Away', 'Favorite', 'Score', 'Spread', '',
            'Underdog', 'Over/Under', 'Notes'
        ])
        df.drop(columns=[''], inplace=True)

        abbr_df = pd.DataFrame(abbr_data, columns=['Favorite_Abbr', 'Underdog_Abbr'])
        df = pd.concat([df, abbr_df], axis=1)
        df['season'] = season
        df['week'] = week

        # Expand to team-level format
        team_rows = []
        for _, row in df.iterrows():
            home_away = row['Home_Away']
            score = row['Score']

            team_rows.extend([
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Favorite_Abbr'],
                    'opponent_abbr': row['Underdog_Abbr'],
                    'home': home_away != '@', 'role': 'Favorite',
                    'result': score.split()[0] if score else None,
                    'score': score.split()[1] if score else None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                },
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Underdog_Abbr'],
                    'opponent_abbr': row['Favorite_Abbr'],
                    'home': home_away == '@', 'role': 'Underdog',
                    'result': None, 'score': None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                }
            ])

        weekly_df = pd.DataFrame(team_rows)

        # Clean the spread column but DO NOT normalize abbreviations yet
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^P.*', '0', regex=True)
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^[WL]\s*', '', regex=True)
        weekly_df['spread'] = pd.to_numeric(weekly_df['spread'].str.strip(), errors='coerce')

        tm_game_odds_regular_team_level_list.append(weekly_df)

In [84]:
# Combine and Export the Regular Season Data
# output: csv file of regular season odds lines

# Combine all regular season data
tm_game_odds_regular_team_level = pd.concat(
    tm_game_odds_regular_team_level_list,
    ignore_index=True
)

# Preview dimensions
print(f"📊 Total rows collected: {tm_game_odds_regular_team_level.shape[0]}")
print(f"✅ Columns: {list(tm_game_odds_regular_team_level.columns)}")

# Export for inspection
# tm_game_odds_regular_team_level.to_csv("tm_game_odds_regular_season.csv", index=False)
# print("📝 Exported to tm_game_odds_regular_season.csv")

📊 Total rows collected: 4222
✅ Columns: ['season', 'week', 'team_abbr', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under']


In [85]:
# extract playoff <h3> tags from each season and pair them with the corresponding soup object
# output: a list of tuples

# Identify playoff <h3> tags for each season 
playoff_h3_tags = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Checking {season} playoffs...")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    tag = soup.find('h3', string=f"{season} Playoffs")
    
    if tag:
        playoff_h3_tags.append((season, soup, tag))
    else:
        print(f"⚠️ No playoff tag found for {season}")

🔄 Checking 2017 playoffs...
🔄 Checking 2018 playoffs...
🔄 Checking 2019 playoffs...
🔄 Checking 2020 playoffs...
🔄 Checking 2021 playoffs...
🔄 Checking 2022 playoffs...
🔄 Checking 2023 playoffs...
🔄 Checking 2024 playoffs...
🔄 Checking 2025 playoffs...
⚠️ No playoff tag found for 2025


In [86]:
# extract all <tr> rows from the playoff table corresponding to each <h3> tag
# output: a list of tuples: (season, [row1, row2, ..., rowN])

# Extract playoff table rows from each <h3> tag
playoff_table_rows = []

for season, soup, tag in playoff_h3_tags:
    print(f"📅 Extracting table rows for {season} playoffs...")
    
    try:
        table = tag.find_next('table')
        rows = table.find('tbody').find_all('tr')
        playoff_table_rows.append((season, rows))
    except Exception as e:
        print(f"⚠️ {season} - Failed to extract table rows: {e}")

📅 Extracting table rows for 2017 playoffs...
📅 Extracting table rows for 2018 playoffs...
📅 Extracting table rows for 2019 playoffs...
📅 Extracting table rows for 2020 playoffs...
📅 Extracting table rows for 2021 playoffs...
📅 Extracting table rows for 2022 playoffs...
📅 Extracting table rows for 2023 playoffs...
📅 Extracting table rows for 2024 playoffs...


In [87]:
# Parse each playoff row into game-level dictionaries
# output: dataframe of the playoff games

# Parse <tr> rows into dictionaries with playoff data
def parse_playoff_row(row):
    try:
        cells = row.find_all('td')
        if len(cells) < 11:
            return None  # Skip malformed rows

        # Remove seed info from names (e.g., "BUF (3)" → "BUF")
        favorite_raw = cells[5].get_text(strip=True).rsplit('(', 1)[0].strip()
        underdog_raw = cells[9].get_text(strip=True).rsplit('(', 1)[0].strip()

        return {
            'Round': cells[0].get_text(strip=True),
            'Day': cells[1].get_text(strip=True),
            'Date': cells[2].get_text(strip=True),
            'Time (ET)': cells[3].get_text(strip=True),
            'Home_Away': cells[4].get_text(strip=True),
            'Favorite': favorite_raw,
            'Score': cells[6].get_text(strip=True),
            'Spread': cells[7].get_text(strip=True),
            'Underdog': underdog_raw,
            'Over/Under': cells[10].get_text(strip=True)
        }

    except Exception as e:
        print(f"⚠️ Error parsing row: {e}")
        return None

# Apply the parser
parsed_playoff_games = []

for season, rows in playoff_table_rows:
    parsed = [parse_playoff_row(row) for row in rows]
    parsed = [game for game in parsed if game is not None]
    for game in parsed:
        game['season'] = season  # Tag season
    parsed_playoff_games.extend(parsed)

# Preview the first few
pd.DataFrame(parsed_playoff_games).head()

,Round,Day,Date,Time (ET),Home_Away,Favorite,Score,Spread,Underdog,Over/Under,season
0,AFC Wild Card,Sat,"Jan 6, 2018",4:35,@,Kansas City Chiefs,L 21-22,L -8.5,Tennessee Titans,U 44,2017
1,NFC Wild Card,Sat,"Jan 6, 2018",8:15,@,Los Angeles Rams,L 13-26,L -6.5,Atlanta Falcons,U 48,2017
2,AFC Wild Card,Sun,"Jan 7, 2018",1:05,@,Jacksonville Jaguars,W 10-3,L -8,Buffalo Bills,U 39.5,2017
3,NFC Wild Card,Sun,"Jan 7, 2018",4:40,@,New Orleans Saints,W 31-26,L -6.5,Carolina Panthers,O 47.5,2017
4,NFC Divisional,Sat,"Jan 13, 2018",4:35,,Atlanta Falcons,L 10-15,L -2.5,Philadelphia Eagles,U 40,2017


In [88]:
# Convert parsed playoff games into team-level rows
# output: dataframe of the playoff games with numbered weeks

# Helper to map playoff round name → week offset
def get_playoff_week(round_name, season):
    base_week = 19 if season >= 2021 else 18
    round_name = round_name.lower()
    if "wild card" in round_name:
        return base_week
    elif "divisional" in round_name:
        return base_week + 1
    elif "championship" in round_name:
        return base_week + 2
    elif "super bowl" in round_name:
        return base_week + 3
    return None  # fallback

# Convert each game into two team rows
team_level_rows = []

for game in parsed_playoff_games:
    week = get_playoff_week(game['Round'], game['season'])

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Favorite'],
        'opponent_abbr': game['Underdog'],
        'home': game['Home_Away'] != '@',
        'role': 'Favorite',
        'result': game['Score'].split()[0] if game['Score'] else None,
        'score': game['Score'].split()[1] if game['Score'] else None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Underdog'],
        'opponent_abbr': game['Favorite'],
        'home': game['Home_Away'] == '@',
        'role': 'Underdog',
        'result': None,
        'score': None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

# Build team-level playoff DataFrame
tm_game_odds_playoffs_team_level = pd.DataFrame(team_level_rows)

# Clean 'spread' values
tm_game_odds_playoffs_team_level['spread'] = (
    tm_game_odds_playoffs_team_level['spread']
    .str.replace(r'^P.*', '0', regex=True)
    .str.replace(r'^[WL]\s*', '', regex=True)
    .str.strip()
)
tm_game_odds_playoffs_team_level['spread'] = pd.to_numeric(
    tm_game_odds_playoffs_team_level['spread'], errors='coerce'
)

# Preview
display(tm_game_odds_playoffs_team_level.head())

,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,18,Kansas City Chiefs,Tennessee Titans,False,Favorite,L,21-22,-8.5,U 44
1,2017,18,Tennessee Titans,Kansas City Chiefs,True,Underdog,None,None,-8.5,U 44
2,2017,18,Los Angeles Rams,Atlanta Falcons,False,Favorite,L,13-26,-6.5,U 48
3,2017,18,Atlanta Falcons,Los Angeles Rams,True,Underdog,None,None,-6.5,U 48
4,2017,18,Jacksonville Jaguars,Buffalo Bills,False,Favorite,W,10-3,-8.0,U 39.5


In [89]:
# Merge Regular Season + Playoffs into One DataFrame
# output: combined dataframe of regular season and playoff odds and lines *(not final)
# output: csv file for inspection only *not final

# Combine regular season and playoff team-level rows
tm_game_odds_all_games = pd.concat(
    [tm_game_odds_regular_team_level, tm_game_odds_playoffs_team_level],
    ignore_index=True
)

# Optional: sort for easier inspection
tm_game_odds_all_games.sort_values(by=['season', 'week'], inplace=True)

# Export final combined dataset
# tm_game_odds_all_games.to_csv("tm_game_odds_all_games.csv", index=False)

# Preview
print("✅ Combined all regular season and playoff games.")
print(f"📊 Final shape: {tm_game_odds_all_games.shape}")
display(tm_game_odds_all_games.head())

✅ Combined all regular season and playoff games.
📊 Final shape: (4418, 10)


,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NEP,KCC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KCC,NEP,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48


In [90]:
# standardize team abbreviations
# output: none

# Create One Unified Map
# Canonical full team name → abbreviation
full_name_to_abbr = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LA', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS', 'Washington Football Team': 'WAS'

}

# Abbreviation fix-ups
abbr_fix_map = {
    'ARZ': 'ARI', 'TBB': 'TB', 'NEP': 'NE', 'GBP': 'GB', 'KCC': 'KC',
    'SFF': 'SF', 'LAR': 'LA', 'NOS': 'NO', 'JAC': 'JAX', 'LVR': 'LV'
}

In [91]:
# apply the correct team abbreviations and rename the dataframe
# output: no output

# Rename to final dataframe name for clarity
tm_reg_playoffs_game_lines_odds = tm_game_odds_all_games.copy()

# Apply full name and abbreviation corrections
tm_reg_playoffs_game_lines_odds['team_abbr'] = (
    tm_reg_playoffs_game_lines_odds['team_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

tm_reg_playoffs_game_lines_odds['opponent_abbr'] = (
    tm_reg_playoffs_game_lines_odds['opponent_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

In [92]:
# Compare the team abbreviations: python nfl df to the tm odds df - the sets should match!
# output: list of team abbreviation  

# Canonical set from the base dataset
baseline_nfl_tm_abbr = set(te_ids_ngs_pfr_stats_sorted['recent_team'].dropna().unique())

# Abbreviation sets from the cleaned odds dataframe
home_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['team_abbr'].dropna().unique())
away_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['opponent_abbr'].dropna().unique())

# Combined odds abbreviations
odds_abbrs = home_odds_lines_abbr.union(away_odds_lines_abbr)

# Comparison
only_in_te_ids = baseline_nfl_tm_abbr - odds_abbrs
only_in_odds = odds_abbrs - baseline_nfl_tm_abbr

# Output
print("✅ Unique team_abbr values (home):", sorted(home_odds_lines_abbr))
print()

print("✅ Unique opponent_abbr values (away):", sorted(away_odds_lines_abbr))
print()

print("❌ Abbreviations only in te_ids_ngs_pfr_stats_sorted:", only_in_te_ids)
print("❌ Abbreviations only in tm_reg_playoffs_game_lines_odds:", only_in_odds)

✅ Unique team_abbr values (home): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

✅ Unique opponent_abbr values (away): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

❌ Abbreviations only in te_ids_ngs_pfr_stats_sorted: set()
❌ Abbreviations only in tm_reg_playoffs_game_lines_odds: set()


In [93]:
# **** Final Dataframe of team odds ***

# output: final dataframe of team odds and lines with the csv file

# Save the file (uncomment to write to disk)
# tm_reg_playoffs_game_lines_odds.to_csv("tm_reg_playoffs_game_lines_odds.csv", index=False)

# Display the first 10 rows
display(tm_reg_playoffs_game_lines_odds.head(10))

,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NE,KC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KC,NE,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48
5,2017,1,CHI,ATL,False,Underdog,None,None,-6.5,U 48
6,2017,1,CIN,BAL,False,Favorite,L,0-20,-2.5,U 41.5
7,2017,1,BAL,CIN,True,Underdog,None,None,-2.5,U 41.5
8,2017,1,PIT,CLE,True,Favorite,W,21-18,-10.0,U 47
9,2017,1,CLE,PIT,False,Underdog,None,None,-10.0,U 47


In [94]:
## End: webscraping sportsoddshistory betting lines data ###

In [95]:
## Begin: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##
# ** Files must be in the local directory ** NFL-20xx-DFS-Dataset.xlsx

In [96]:
# create a helper function to clean the the dfs salary data
# no output

def clean_column_dfs(col):
    """
    Cleans and flattens multi-index column names for DFS salary Excel files:
    - Joins tuples if multi-index
    - Removes special characters
    - Normalizes spaces
    - Converts to lowercase for matching
    """
    if isinstance(col, tuple):
        col = ' '.join(str(x) for x in col if x)

    return (
        str(col)
        .replace('\n', ' ')
        .replace('(', '')
        .replace(')', '')
        .replace('"', '')
        .replace('#', '')
        .replace('$', '')
        .replace('/', '')
        .replace('-', ' ')
        .strip()
        .lower()
        .replace('  ', ' ')
        .replace('   ', ' ')
    )

In [97]:
# Read the excel files
filepath = 'NFL-2024-DFS-Dataset.xlsx'
dfs_raw = pd.read_excel(filepath, header=[0, 1])
original_row_count = len(dfs_raw)

dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]
dfs_raw.head()  # Optional preview

,game information bigdataball dataset,game information game id,game information date,game information week,game information start time et,game information player id,game information player dst,game information team,game information opponent,game information venue rh,position draftkings,position fanduel,salary for draftkings classic contests,salary for fanduel full roster contests,fantasy points scored draftkings,fantasy points scored fanduel
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#N/A MEANING: The game was not included in any...,NaN,NaN,NaN,NaN,NaN
1,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,BAL,Baltimore Ravens,Baltimore Ravens,Kansas City Chiefs,Road,DST,D,2700,3900.0,4.0,4.0
2,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,nelson-agholor,Nelson Agholor,Baltimore Ravens,Kansas City Chiefs,Road,WR,WR,3800,4200.0,1.6,1.1
3,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,derrick-henry,Derrick Henry,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,6900,7900.0,10.6,10.6
4,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,patrick-ricard,Patrick Ricard,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,4000,4100.0,0.0,0.0


In [98]:
# helper function does the following:
# Fanduel and Draft Kings player salary data for all positions (QB, RB, TE, WR, DST)
# creates and combines the dataframes for years 2017 - present 
# performs data validation checks
# output: none

def process_single_year(filepath, year):
    
    # Step 1: Read and clean the headers
    dfs_raw = pd.read_excel(filepath, header=[0, 1])
    original_row_count = len(dfs_raw)
    dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]

    # ✅ Step 2: Extract only relevant columns using cleaned names
    expected_cols = {
        'player': 'game information player dst',
        'week': 'game information week',
        'date': 'game information date',
        'player_id': 'game information player id',
        'dk_position': 'position draftkings',
        'fd_position': 'position fanduel',
        'dk_salary': 'salary for draftkings classic contests',
        'fd_salary': 'salary for fanduel full roster contests'
    }

    # Subset the dataframe using cleaned column names
    dfs_subset = dfs_raw[list(expected_cols.values())].copy()

    # Rename them to simple identifiers for internal use
    dfs_subset.columns = list(expected_cols.keys())

    
    dfs_subset['date'] = pd.to_datetime(dfs_subset['date'])

    team_abbreviation_mapping = {
        'NWE': 'NE',
        'SFO': 'SF',
        'OAK': 'LV',
        'KAN': 'KC',
        'TAM': 'TB',
        'NOR': 'NO',
        'LAR': 'LA',
        'GNB': 'GB'
    }
    mask_dst = dfs_subset['dk_position'] == 'DST'
    dfs_subset.loc[mask_dst, 'player_id'] = dfs_subset.loc[mask_dst, 'player_id'].replace(team_abbreviation_mapping)

    def fix_season(row):
        game_year = row['date'].year
        game_month = row['date'].month
        game_week = row['week']
        
        if game_month in [1, 2]:
            if (game_year <= 2020 and game_week >= 18):
                return game_year - 1
            elif (game_year >= 2021 and game_week >= 19):
                return game_year - 1
            elif (game_year >= 2021 and game_week == 18):
                return game_year - 1
        return game_year

    dfs_subset['season'] = dfs_subset.apply(fix_season, axis=1)

    # 🔥 Track NaNs before dropping
    season_nulls_before = dfs_subset['season'].isna().sum()

    dfs_subset = dfs_subset.dropna(subset=['season'])
    dfs_subset['season'] = dfs_subset['season'].astype(int)

    season_nulls_after = dfs_subset['season'].isna().sum()

    print(f"🔎 Season NaN rows dropped: {season_nulls_before}")
    print(f"Remaining NaN rows (should be 0): {season_nulls_after}")

    dfs_subset = dfs_subset.drop(columns=['date'])

    dfs_subset['dk_salary'] = pd.to_numeric(dfs_subset['dk_salary'], errors='coerce')
    dfs_subset['fd_salary'] = pd.to_numeric(dfs_subset['fd_salary'], errors='coerce')
    dfs_subset = dfs_subset.dropna(subset=['dk_salary', 'fd_salary'])
    dfs_subset['dk_salary'] = dfs_subset['dk_salary'].astype(int)
    dfs_subset['fd_salary'] = dfs_subset['fd_salary'].astype(int)
    dfs_subset['week'] = dfs_subset['week'].astype(int)
    
    dfs_subset = dfs_subset[['season', 'week', 'player_id', 'player', 'dk_position', 'fd_position', 'dk_salary', 'fd_salary']]
    
    unique_weeks = dfs_subset['week'].nunique()
    min_week = dfs_subset['week'].min()
    max_week = dfs_subset['week'].max()
    expected_weeks = 21 if int(year) <= 2020 else 22

    print(f"\nProcessing file: {filepath}")
    print(f"Original rows in xlsx file: {original_row_count}")
    print(f"Number of players with no salary data found in xlsx: {original_row_count - len(dfs_subset)}")
    print(f"Rows in csv file after dropping NaNs: {len(dfs_subset)}")

    if original_row_count - (original_row_count - len(dfs_subset)) == len(dfs_subset):
        print("✅ Salary Validation passed: Counts match after dropping NaNs.")
        salary_validation = 'Passed'
    else:
        print("❌ Salary Validation failed: Counts mismatch!")
        salary_validation = 'Failed'

    print(f"Weeks detected: {min_week} to {max_week}")
    print(f"Total unique weeks found: {unique_weeks}")
    print("🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.")

    if unique_weeks == expected_weeks or unique_weeks == expected_weeks - 1:
        print(f"✅ Week Validation passed: {unique_weeks} weeks found (expected {expected_weeks}).\n")
        week_validation = 'Passed'
    else:
        print(f"❌ Week Validation failed: {unique_weeks} weeks found, expected {expected_weeks}.\n")
        week_validation = 'Failed'
    
    return dfs_subset, {
        'year': int(year),
        'original_rows': original_row_count,
        'nan_rows': original_row_count - len(dfs_subset),
        'rows_after_drop': len(dfs_subset),
        'min_week': min_week,
        'max_week': max_week,
        'unique_weeks': unique_weeks,
        'expected_weeks': expected_weeks,
        'salary_validation': salary_validation,
        'week_validation': week_validation
    }

In [99]:
# ** dataframe of Fanduel and Draft Kings Salaries FOR all positions ** 

# main control flow implements the helper function 
# output: combined dataframe and csv files of all seasons fanduel draft kings player salary data 
# output: data validation checks

# Find all matching files
file_list = sorted(glob.glob('NFL-*-DFS-Dataset.xlsx'))

# Handle if no files found
if not file_list:
    print("❌ No xlsx files detected.\nPlease download and place the BigDataBall NFL DFS Excel files into the same directory as this Jupyter Notebook file.")
else:
    # Process each file
    all_years_dfs = []
    validation_records = []
    file_years = []

    for file in file_list:
        year = file.split('-')[1]  # Extract year from filename
        file_years.append(int(year))
        
        year_df, validation_info = process_single_year(file, year)

        # ** csv file ***
        # Save per-year CSV
        # year_df.to_csv(f'nfl_fd_dk_salary_{year}.csv', index=False)
        
        # Append to master list
        all_years_dfs.append(year_df)
        validation_records.append(validation_info)

    # Create validation summary DataFrame
    validation_summary_df = pd.DataFrame(validation_records)
    print("\n📋 Validation Summary:")
    display(validation_summary_df)

    # Combine all years into one big dataframe
    nfl_fd_dk_salary_combined = pd.concat(all_years_dfs, ignore_index=True)

    # Determine latest season dynamically
    current_season = max(file_years)

    # Export final combined CSV
    final_filename = f'nfl_fd_dk_salary_2017_{current_season}.csv'

    # If the file already exists, create a backup
    if os.path.exists(final_filename):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f'nfl_fd_dk_salary_2017_{current_season}_backup_{timestamp}.csv'
        shutil.copy(final_filename, backup_filename)
        print(f"🛡️ Backup created: {backup_filename}")


    # *** csv file ***
    # nfl_fd_dk_salary_combined.to_csv(final_filename, index=False)

    print(f"\n✅ Final combined CSV saved as: {final_filename}")

    # Display a quick preview
    display(nfl_fd_dk_salary_combined.head())

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2017-DFS-Dataset.xlsx
Original rows in xlsx file: 7664
Number of players with no salary data found in xlsx: 799
Rows in csv file after dropping NaNs: 6865
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found (expected 21).

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2018-DFS-Dataset.xlsx
Original rows in xlsx file: 7216
Number of players with no salary data found in xlsx: 197
Rows in csv file after dropping NaNs: 7019
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found 

,year,original_rows,nan_rows,rows_after_drop,min_week,max_week,unique_weeks,expected_weeks,salary_validation,week_validation
0,2017,7664,799,6865,1,20,20,21,Passed,Passed
1,2018,7216,197,7019,1,20,20,21,Passed,Passed
2,2019,7252,268,6984,1,20,20,21,Passed,Passed
3,2020,7572,385,7187,1,20,20,21,Passed,Passed
4,2021,7994,1650,6344,1,21,21,22,Passed,Passed
5,2022,7872,138,7734,1,21,21,22,Passed,Passed
6,2023,7897,65,7832,1,21,21,22,Passed,Passed
7,2024,7912,136,7776,1,21,21,22,Passed,Passed



✅ Final combined CSV saved as: nfl_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
0,2017,1,KC,Kansas City Chiefs,DST,DST,2300,4300
1,2017,1,NE,New England Patriots,DST,DST,3400,4500
2,2017,1,NaN,Alex Smith,QB,QB,5400,6900
3,2017,1,NaN,Tom Brady,QB,QB,7600,8900
4,2017,1,NaN,Anthony Sherman,RB,RB,3000,4500


In [100]:
# team abbreviations validations check
def validate_team_defenses(dfs_df, year_label, baseline_set):
    dfs_team_defense = dfs_df.loc[dfs_df['dk_position'] == 'DST', 'player_id']
    dfs_nfl_tm_abbr = set(dfs_team_defense.dropna().unique())
    
    difference_dfs = dfs_nfl_tm_abbr - baseline_set
    difference_baseline = baseline_set - dfs_nfl_tm_abbr
    
    print(f"\nValidating Team Defenses for {year_label}:")
    if not difference_dfs and not difference_baseline:
        print(f"✅ Team defenses match for {year_label}")
    else:
        print(f"❌ Team mismatch detected for {year_label}")
        print(f"Teams only in DFS: {difference_dfs}")
        print(f"Teams only in Baseline: {difference_baseline}")

# Validate each yearly dataframe
for df, info in zip(all_years_dfs, validation_records):
    validate_team_defenses(df, year_label=info['year'], baseline_set=baseline_nfl_tm_abbr)

# Validate the full combined dataframe
validate_team_defenses(nfl_fd_dk_salary_combined, year_label='Combined', baseline_set=baseline_nfl_tm_abbr)


Validating Team Defenses for 2017:
✅ Team defenses match for 2017

Validating Team Defenses for 2018:
✅ Team defenses match for 2018

Validating Team Defenses for 2019:
✅ Team defenses match for 2019

Validating Team Defenses for 2020:
✅ Team defenses match for 2020

Validating Team Defenses for 2021:
✅ Team defenses match for 2021

Validating Team Defenses for 2022:
✅ Team defenses match for 2022

Validating Team Defenses for 2023:
✅ Team defenses match for 2023

Validating Team Defenses for 2024:
✅ Team defenses match for 2024

Validating Team Defenses for Combined:
✅ Team defenses match for Combined


In [101]:
# ** Final TE dataframe of Fanduel and Draft Kings player Salaries ** 

# output: saved and previewed TE salary dataframe

# Determine current season based on available data
current_season = nfl_fd_dk_salary_combined['season'].max()

# Extract TE players where DraftKings position is TE
te_fd_dk_salary_2017_current_df = nfl_fd_dk_salary_combined.loc[
    nfl_fd_dk_salary_combined['dk_position'] == 'TE'
]

# *** csv file ***
te_csv_filename = f'te_fd_dk_salary_2017_{current_season}.csv'
# te_fd_dk_salary_2017_current_df.to_csv(te_csv_filename, index=False)

print(f"✅ TE DFS dataframe created and saved as {te_csv_filename}")

# Optional: Display a quick preview
display(te_fd_dk_salary_2017_current_df.head())

✅ TE DFS dataframe created and saved as te_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
13,2017,1,NaN,Demetrius Harris,TE,TE,2500,4500
14,2017,1,NaN,Ross Travis,TE,TE,2500,4500
15,2017,1,NaN,Travis Kelce,TE,TE,5600,6500
16,2017,1,NaN,Dwayne Allen,TE,TE,2700,5000
17,2017,1,NaN,Rob Gronkowski,TE,TE,6800,8000


In [102]:
## End: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##

In [103]:
### Begin: Data Normalization and Merge Process ###

In [104]:
## Dataframes ##
# te_ids_ngs_pfr_stats_sorted
# te_adv_fpts_rost_rz_merged_df_sorted_hash (not going to use this one)
# tm_reg_playoffs_game_lines_odds
# te_fd_dk_salary_2017_current_df

## Apply Alternative for merging fantasypros ##
# te_fp_advanced_stats_df
# te_fpts_perct_rost_df
# te_redzone_stats_df

In [105]:
# view columns of each fantasypros dataframe prior to normalization
# output: columns in each fp dataframe

print('te_fp_advanced_stats_df')
print(te_fp_advanced_stats_df.columns.tolist())
print()

print('te_fpts_perct_rost_df')
print(te_fpts_perct_rost_df.columns.tolist())
print()

print('te_redzone_stats_df')
print(te_redzone_stats_df.columns.tolist())

te_fp_advanced_stats_df
['Year', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'REC', 'YDS', 'YBC', 'AIR', 'YAC', 'YACON', 'BRKTKL', 'TGT', 'CATCHABLE', 'DROP', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS']

te_fpts_perct_rost_df
['FantasyPros_ID', 'FPTS', 'ROST', 'Year', 'Week']

te_redzone_stats_df
['FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'Year', 'Week']


In [106]:
# helper function for normalization methods
def normalize_dataframe(df: pd.DataFrame, type_map: dict = None) -> pd.DataFrame:
    """
    Clean and normalize dataframe:
    - Strip whitespace from object columns
    - Lowercase common ID/name fields
    - Replace special NA tokens with np.nan
    - Cast to types from provided type_map
    """
    df = df.copy()

    # Replace common string-based missing values with np.nan
    df.replace(["N/A", "NA", "-", ""], np.nan, inplace=True)

    # Strip whitespace from string/object columns
    for col in df.select_dtypes(include='object').columns:
        try:
            df[col] = df[col].astype(str).str.strip()
        except Exception as e:
            print(f"⚠️ Could not strip column '{col}': {e}")

    # Lowercase likely ID/name fields (if present)
    for key in ['player', 'player_name', 'player_id', 'fantasypros_id', 'FantasyPros_ID', 'merge_name']:
        if key in df.columns:
            df[key] = df[key].astype(str).str.lower()

    # Apply type conversions as defined in type_map
    if type_map:
        for col, dtype in type_map.items():
            if col in df.columns:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    print(f"⚠️ Warning: could not convert column '{col}' to {dtype}. Reason: {e}")

    return df

In [107]:
def clean_percentage_columns(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        if col in df.columns:
            try:
                # Convert to string first (safe for NaNs)
                df[col] = df[col].astype(str).str.replace('%', '', regex=False)
                df[col] = df[col].astype(float)
            except Exception as e:
                print(f"⚠️ Could not clean and convert '{col}': {e}")
    return df

In [108]:
def inspect_dataframe_types(df: pd.DataFrame, name: str = "DataFrame") -> None:
    print(f"📋 Inspecting: {name}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    summary = pd.DataFrame({
        'dtype': df.dtypes,
        'na_count': df.isna().sum()
    }).sort_index()
    display(summary)

In [109]:
# TE Baseline type map
type_map_te = {
    'season': 'int32',
    'week': 'int32',
    'player_id': 'string',
    'fantasypros_id': 'string',
}

# FP Advanced Stats type map
type_map_fp_adv_te = {
    'FantasyPros_ID': 'string',
    'Season': 'int32',
    'Week': 'int32',
    'Player': 'string',
    'G': 'int32',
    'REC': 'float32',
    'YDS': 'float32',
    'YBC': 'float32',
    'AIR': 'float32',
    'YAC': 'float32',
    'YACON': 'float32',
    'BRKTKL': 'float32',
    'TGT': 'float32',
    'CATCHABLE': 'float32',
    'DROP': 'float32',
    'RZ TGT': 'float32',
    '10+ YDS': 'float32',
    '20+ YDS': 'float32',
    '30+ YDS': 'float32',
    '40+ YDS': 'float32',
    '50+ YDS': 'float32',
}

# FP Fantasy Points + Rostered type map
type_map_fp_fpts_te = {
    'FantasyPros_ID': 'string',
    'Season': 'int32',
    'Week': 'int32',
    'FPTS': 'float32',
    'ROST': 'string',
}

# FP Redzone type map
type_map_fp_rz_te = {
    'FantasyPros_ID': 'string',
    'Season': 'int32',
    'Week': 'int32',
    'REC PCT_rz': 'float32',
    'REC_rz': 'int32',
    'TGT PCT_rz': 'float32',
    'TGT_rz': 'int32',
    'Y/R_rz': 'float32',
}


In [110]:
# apply the normalization function to the baseline NFL Python stats dataframe
# no output
te_ids_ngs_pfr_stats_sorted_normalized = normalize_dataframe(
    te_ids_ngs_pfr_stats_sorted,
    type_map_te
)

# Fix FP ID formatting
te_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id'] = (
    te_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id']
    .astype(str).str.replace(r'\.0$', '', regex=True)
)

In [111]:
# Rename merge keys consistently BEFORE normalization to the fantasypros dataframes
# no output
te_fp_advanced_stats_df = te_fp_advanced_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

te_fpts_perct_rost_df = te_fpts_perct_rost_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

te_redzone_stats_df = te_redzone_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

# Remove duplicate 'Player' column if it exists
te_fp_advanced_stats_df = te_fp_advanced_stats_df.loc[:, ~te_fp_advanced_stats_df.columns.duplicated()]

# Clean percentages BEFORE normalization
te_redzone_stats_df = clean_percentage_columns(
    te_redzone_stats_df,
    ['REC PCT_rz', 'TGT PCT_rz']
)

In [112]:
# Apply the type_map data normalizations to the fantasypros dataframes
# no output

te_fp_advanced_stats_df = normalize_dataframe(
    te_fp_advanced_stats_df,
    type_map_fp_adv_te
)

te_fpts_perct_rost_df = normalize_dataframe(
    te_fpts_perct_rost_df,
    type_map_fp_fpts_te
)

te_redzone_stats_df = normalize_dataframe(
    te_redzone_stats_df,
    type_map_fp_rz_te
)


In [113]:
# Standardize all column names to lowercase after normalization
te_fp_advanced_stats_df.columns = te_fp_advanced_stats_df.columns.str.lower()
te_fpts_perct_rost_df.columns = te_fpts_perct_rost_df.columns.str.lower()
te_redzone_stats_df.columns = te_redzone_stats_df.columns.str.lower()

In [114]:
# view columns of each fantasypros dataframe after normalization
# output: columns in each fp dataframe

print('te_fp_advanced_stats_df')
print(te_fp_advanced_stats_df.columns.tolist())
print()

print('te_fpts_perct_rost_df')
print(te_fpts_perct_rost_df.columns.tolist())
print()

print('te_redzone_stats_df')
print(te_redzone_stats_df.columns.tolist())

te_fp_advanced_stats_df
['season', 'week', 'fantasypros_id', 'player', 'g', 'rec', 'yds', 'ybc', 'air', 'yac', 'yacon', 'brktkl', 'tgt', 'catchable', 'drop', 'rz tgt', '10+ yds', '20+ yds', '30+ yds', '40+ yds', '50+ yds']

te_fpts_perct_rost_df
['fantasypros_id', 'fpts', 'rost', 'season', 'week']

te_redzone_stats_df
['fantasypros_id', 'rec_rz', 'tgt_rz', 'rec pct_rz', 'y/r_rz', 'tgt pct_rz', 'season', 'week']


In [115]:
# Merge the NFL Python Stats and FantasyPros dataframes
# no output

# Start from the normalized baseline
te_nfl_py_fp_merged = te_ids_ngs_pfr_stats_sorted_normalized.copy()

# Merge: Advanced stats
te_nfl_py_fp_merged = te_nfl_py_fp_merged.merge(
    te_fp_advanced_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_adv')
)

# Merge: FPTS + ROST
te_nfl_py_fp_merged = te_nfl_py_fp_merged.merge(
    te_fpts_perct_rost_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_fpts')
)

# Merge: Redzone stats
te_nfl_py_fp_merged = te_nfl_py_fp_merged.merge(
    te_redzone_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_rz')
)

In [116]:
# ** Merged Dataframe: NFL Python Stats and FantasyPros

# output: merged nfl py and fp validation stats
# output: merged nfl py and fp dataframe and csv file

print(f"📊 Final merged shape: {te_nfl_py_fp_merged.shape}")

print(f"📦 FP Advanced Stats Columns Merged: {te_fp_advanced_stats_df.shape[1] - 3} / {te_fp_advanced_stats_df.shape[1] - 3}")
print(f"📦 FP Fantasy Points & ROST Columns Merged: 2 / 2")
print(f"📦 FP Redzone Columns Merged: {te_redzone_stats_df.shape[1] - 3} / {te_redzone_stats_df.shape[1] - 3}")

# Calculate merge coverage
adv_merged = te_nfl_py_fp_merged[te_fp_advanced_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
fpts_merged = te_nfl_py_fp_merged[te_fpts_perct_rost_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
rz_merged = te_nfl_py_fp_merged[te_redzone_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]

print(f"📈 FP Advanced Stats coverage: {adv_merged} rows ({adv_merged / len(te_fp_advanced_stats_df) * 100:.2f}%)")
print(f"📈 FP FPTS/ROST coverage: {fpts_merged} rows ({fpts_merged / len(te_fpts_perct_rost_df) * 100:.2f}%)")
print(f"📈 FP Redzone Stats coverage: {rz_merged} rows ({rz_merged / len(te_redzone_stats_df) * 100:.2f}%)")

# Display first 10 rows
display(te_nfl_py_fp_merged.head(10))

# *** csv file ***
# te_nfl_py_fp_merged.to_csv("te_nfl_py_fp_merged.csv", index=False)
# print("📁 Exported: te_nfl_py_fp_merged.csv")

📊 Final merged shape: (8782, 94)
📦 FP Advanced Stats Columns Merged: 18 / 18
📦 FP Fantasy Points & ROST Columns Merged: 2 / 2
📦 FP Redzone Columns Merged: 5 / 5
📈 FP Advanced Stats coverage: 1110 rows (99.11%)
📈 FP FPTS/ROST coverage: 8408 rows (55.53%)
📈 FP Redzone Stats coverage: 3425 rows (97.97%)


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat,player,g,rec,yds,ybc,air,yac,yacon,brktkl,tgt,catchable,drop,rz tgt,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,fpts,rost,rec_rz,tgt_rz,rec pct_rz,y/r_rz,tgt pct_rz
0,2017,REG,1,00-0032392,a.hooper,TE,TE,ATL,12677,90c2a93f-d837-4e1b-b57c-56648903a8db,15581,10715.0,3202.0,nan,3043275.0,29315.0,NaN,2067004.0,HoopAu00,austin-hooper-1,10748.0,11301.0,NaN,29315.0,739424.0,17963.0,739424.0,Austin Hooper,austin hooper,2,2,128.0,1,0.0,0.0,45.0,83.0,2.0,11.335708,0,2.844445,0.066667,0.208333,0.245833,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Austin Hooper,1.0,2.0,128.0,45.0,0.0,83.0,65.0,3.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,1.0,18.799999,3.5%,NaN,NaN,NaN,NaN,NaN
1,2017,REG,1,00-0030061,z.ertz,TE,TE,PHI,11247,de3421f7-2147-4835-89a5-724e87bad463,11689,7816.0,1339.0,zachertz/2540158,15835.0,26658.0,NaN,1685963.0,ErtzZa00,zach-ertz-1,8781.0,8409.0,305.0,26658.0,503177.0,14856.0,503177.0,Zach Ertz,zach ertz,8,8,93.0,0,0.0,0.0,78.0,15.0,6.0,7.477927,0,1.192308,0.210526,0.194514,0.451949,Zach Ertz,PHI,4.50000,2.710968,9.355000,18.126332,100.000000,93.0,0.0,2.270000,2.177001,0.092999,00-0030061,Zachary,Ertz,Z.Ertz,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Zach Ertz,1.0,8.0,93.0,78.0,0.0,15.0,5.0,0.0,8.0,8.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,9.300000,59.2%,NaN,NaN,NaN,NaN,NaN
2,2017,REG,1,00-0024389,de.walker,TE,TE,TEN,8416,ccce5e8e-52ca-4f0f-a40f-fe5e7227d156,9172,3120.0,284.0,delaniewalker/2495966,9761.0,7924.0,4353.0,1109396.0,WalkHu00,nan,4888.0,3976.0,NaN,7924.0,218943.0,7175.0,NaN,Delanie Walker,delanie walker,7,9,76.0,0,0.0,0.0,104.0,12.0,5.0,5.525133,0,0.730769,0.219512,0.279570,0.524967,Delanie Walker,TEN,7.57250,3.097148,10.901111,26.669023,77.777778,76.0,0.0,2.358571,3.025790,-0.667219,00-0024389,Delanie,Walker,D.Walker,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Delanie Walker,1.0,7.0,76.0,67.0,0.0,9.0,4.0,0.0,9.0,7.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,7.600000,0.1%,NaN,NaN,NaN,NaN,NaN
3,2017,REG,1,00-0031299,j.matthews,TE,TE,BUF,11676,7b96a836-666b-47b6-a0a7-9dbb0b4c53e8,12124,8677.0,1800.0,jordanmatthews/2543500,16763.0,27570.0,NaN,1759816.0,MattJo00,jordan-matthews-1,9273.0,9420.0,NaN,27570.0,555648.0,15974.0,555648.0,Jordan Matthews,jordan matthews,2,3,61.0,0,0.0,0.0,56.0,42.0,2.0,5.050018,0,1.089286,0.111111,0.306011,0.380874,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0%,NaN,NaN,NaN,NaN,NaN
4,2017,REG,1,00-0022127,j.witten,TE,TE,DAL,6997,e38c9b1b-7c51-48a2-ac1d-a752502e8930,9275,1384.0,23.0,jasonwitten/2505629,4527.0,6405.0,NaN,396134.0,WittJa00,jason-witten-1,3086.0,1990.0,NaN,6405.0,184571.0,722.0,NaN,Jason Witten,jason witten,7,9,59.0,1,0.0,0.0,61.0,12.0,1.0,2.561272,0,0.967213,0.236842,0.205387,0.499034,Jason Witten,DAL,4.08200,1.552612,6.372222,18.549065,77.777778,59.0,1.0,1.725714,1.558290,0.167425,00-0022127,Christopher,Witten,J.Witten,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Jason Witten,1.0,7.0,59.0,47.0,0.0,12.0,4.0,0.0,9.0,7.0,0.0,1.0,3.0

In [117]:
# compare match rates using different columns
# output: match rates show you should join the 'recent_team' column in the baseline df

# Option A: merge on 'team_abbr'
merged_team_abbr = te_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'team_abbr'],
    right_on=['season', 'week', 'team_abbr']
)

# Option B: merge on 'recent_team'
merged_recent_team = te_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr']  # same column name on right
)

# Count matches for each
matched_abbr = merged_team_abbr['spread'].notna().sum()
matched_recent = merged_recent_team['spread'].notna().sum()
total_rows = len(te_nfl_py_fp_merged)

# Calculate match rates
rate_abbr = round(100 * matched_abbr / total_rows, 2)
rate_recent = round(100 * matched_recent / total_rows, 2)

print(f"Match rate using team_abbr:     {matched_abbr} / {total_rows} rows matched ({rate_abbr}%)")
print(f"Match rate using recent_team:   {matched_recent} / {total_rows} rows matched ({rate_recent}%)")

Match rate using team_abbr:     2388 / 8782 rows matched (27.19%)
Match rate using recent_team:   8761 / 8782 rows matched (99.76%)


In [118]:
# *** Dataframe of nfl python stats, fantasypros, and sportsOddsHistory ***
# Merge the nfl python stats and fantasypros dataframe with the odds / lines dataframe
# output: merged dataframe and csv file

te_nfl_py_fp_odds_merged = te_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr'],
    validate='many_to_one'
)

# ✅ Row count check
print("Row check:")
print("Before merge:", len(te_nfl_py_fp_merged))
print("After merge: ", len(te_nfl_py_fp_odds_merged))

# ✅ Column count check
print("\nColumn check:")
print("Before merge:", len(te_nfl_py_fp_merged.columns))
print("After merge: ", len(te_nfl_py_fp_odds_merged.columns))

# ✅ Display sample
print("\nSample rows:")
display(te_nfl_py_fp_odds_merged.head())

# ✅ ** csv file ***
# te_nfl_py_fp_odds_merged.to_csv('te_nfl_py_fp_odds_merged.csv', index=False)


Row check:
Before merge: 8782
After merge:  8782

Column check:
Before merge: 94
After merge:  102

Sample rows:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr_x,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat,player,g,rec,yds,ybc,air,yac,yacon,brktkl,tgt,catchable,drop,rz tgt,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,fpts,rost,rec_rz,tgt_rz,rec pct_rz,y/r_rz,tgt pct_rz,team_abbr_y,opponent_abbr,home,role,result,score,spread,over_under
0,2017,REG,1,00-0032392,a.hooper,TE,TE,ATL,12677,90c2a93f-d837-4e1b-b57c-56648903a8db,15581,10715.0,3202.0,nan,3043275.0,29315.0,NaN,2067004.0,HoopAu00,austin-hooper-1,10748.0,11301.0,NaN,29315.0,739424.0,17963.0,739424.0,Austin Hooper,austin hooper,2,2,128.0,1,0.0,0.0,45.0,83.0,2.0,11.335708,0,2.844445,0.066667,0.208333,0.245833,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Austin Hooper,1.0,2.0,128.0,45.0,0.0,83.0,65.0,3.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,1.0,18.799999,3.5%,NaN,NaN,NaN,NaN,NaN,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48
1,2017,REG,1,00-0030061,z.ertz,TE,TE,PHI,11247,de3421f7-2147-4835-89a5-724e87bad463,11689,7816.0,1339.0,zachertz/2540158,15835.0,26658.0,NaN,1685963.0,ErtzZa00,zach-ertz-1,8781.0,8409.0,305.0,26658.0,503177.0,14856.0,503177.0,Zach Ertz,zach ertz,8,8,93.0,0,0.0,0.0,78.0,15.0,6.0,7.477927,0,1.192308,0.210526,0.194514,0.451949,Zach Ertz,PHI,4.5000,2.710968,9.355000,18.126332,100.000000,93.0,0.0,2.270000,2.177001,0.092999,00-0030061,Zachary,Ertz,Z.Ertz,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Zach Ertz,1.0,8.0,93.0,78.0,0.0,15.0,5.0,0.0,8.0,8.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,9.300000,59.2%,NaN,NaN,NaN,NaN,NaN,PHI,WAS,True,Favorite,W,30-17,-2.0,U 49.5
2,2017,REG,1,00-0024389,de.walker,TE,TE,TEN,8416,ccce5e8e-52ca-4f0f-a40f-fe5e7227d156,9172,3120.0,284.0,delaniewalker/2495966,9761.0,7924.0,4353.0,1109396.0,WalkHu00,nan,4888.0,3976.0,NaN,7924.0,218943.0,7175.0,NaN,Delanie Walker,delanie walker,7,9,76.0,0,0.0,0.0,104.0,12.0,5.0,5.525133,0,0.730769,0.219512,0.279570,0.524967,Delanie Walker,TEN,7.5725,3.097148,10.901111,26.669023,77.777778,76.0,0.0,2.358571,3.025790,-0.667219,00-0024389,Delanie,Walker,D.Walker,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,Delanie Walker,1.0,7.0,76.0,67.0,0.0,9.0,4.0,0.0,9.0,7.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,7.600000,0.1%,NaN,NaN,NaN,NaN,NaN,TEN,LV,False,Favorite,L,16-26,-2.5,U 49.5
3,2017,REG,1,00-0031299,j.matthews,TE,TE,BUF,11676,7b96a836-666b-47b6-a0a7-9dbb0b4c53e8,12124,8677.0,1800.0,jordanmatthews/2543500,16763.0,27570.0,NaN,1759816.0,MattJo00,jordan-matthews-1,9273.0,9420.0,NaN,27570.0,555648.0,15974.0,555648.0,Jordan Matthews,jordan matthews,2,3,61.0,0,0.0,0.0,56.0,42.0,2.0,5.050018,0,1.089286,0.111111,0.306011,0.380874,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0%,NaN,NaN,NaN,NaN,NaN,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
4,2017,REG,1,00-0022127,j.witten,TE,TE,DAL,6997,e38c9b1b-7c51-48a2-ac1d-a752502e8930,9275,1384.0,23.0,jasonwitten/2505629,4527.0,6405.0,NaN,396134.0,WittJa00,jason-witten-1,3086.0,1990.0,NaN,6405.0,184571.0,722.0,NaN,Jason Witten,jason witten,7,9,59.0,1,0.0,0.0,61.0,12.0,1.0,2.561272,0,0.967213,0.236842,0.205387,0.499034,Jason Wit

In [119]:
# create key values for merging and match tests
# output: sample dataframe of matched rows and validation tests

# Standardize and construct merge keys safely ---
te_fd_dk_salary_2017_current_df = te_fd_dk_salary_2017_current_df.copy()
te_fd_dk_salary_2017_current_df.loc[:, 'merge_name'] = (
    te_fd_dk_salary_2017_current_df['player']
    .str.lower()
    .str.replace(r'[^a-z ]', '', regex=True)
)

te_fd_dk_salary_2017_current_df.loc[:, 'merge_key'] = (
    te_fd_dk_salary_2017_current_df['merge_name'] + '_' +
    te_fd_dk_salary_2017_current_df['season'].astype(str) + '_' +
    te_fd_dk_salary_2017_current_df['week'].astype(str)
)

te_nfl_py_fp_odds_merged = te_nfl_py_fp_odds_merged.copy()
te_nfl_py_fp_odds_merged.loc[:, 'merge_key'] = (
    te_nfl_py_fp_odds_merged['merge_name'] + '_' +
    te_nfl_py_fp_odds_merged['season'].astype(str) + '_' +
    te_nfl_py_fp_odds_merged['week'].astype(str)
)

# Inner join for validation ---
cols_salary = ['season', 'week', 'merge_key', 'merge_name', 'dk_salary', 'fd_salary']
test_matches = te_nfl_py_fp_odds_merged.merge(
    te_fd_dk_salary_2017_current_df[cols_salary],
    on=['season', 'week', 'merge_key'],
    how='inner',
    suffixes=('_main', '_salary')
)

# Match statistics
print(f"✅ Inner join matches: {len(test_matches):,}")

num_matches = test_matches['merge_key'].nunique()
total_salary_rows = te_fd_dk_salary_2017_current_df['merge_key'].nunique()
match_percent = (num_matches / total_salary_rows) * 100

print(f"✅ Match rate: {num_matches:,} / {total_salary_rows:,} salary rows matched ({match_percent:.2f}%)")

# --- STEP 4: Display random sample for visual verification ---
sample_matches = test_matches.sample(10, random_state=42)[
    ['season', 'week', 'merge_key', 'merge_name_main', 'merge_name_salary', 'dk_salary', 'fd_salary']
]

display(sample_matches)

✅ Inner join matches: 8,064
✅ Match rate: 8,021 / 12,508 salary rows matched (64.13%)


,season,week,merge_key,merge_name_main,merge_name_salary,dk_salary,fd_salary
742,2017,14,dwayne allen_2017_14,dwayne allen,dwayne allen,2700,4500
2126,2019,3,gerald everett_2019_3,gerald everett,gerald everett,3200,4500
4076,2021,3,jonnu smith_2021_3,jonnu smith,jonnu smith,2500,5200
2941,2019,19,mark andrews_2019_19,mark andrews,mark andrews,5600,6800
3872,2020,16,vance mcdonald_2020_16,vance mcdonald,vance mcdonald,3200,4200
1302,2018,6,nick oleary_2018_6,nick oleary,nick oleary,2500,4000
1168,2018,3,james oshaughnessy_2018_3,james oshaughnessy,james oshaughnessy,2500,4000
5090,2022,6,greg dulcich_2022_6,greg dulcich,greg dulcich,2500,4000
3050,2020,2,mark andrews_2020_2,mark andrews,mark andrews,6300,7500
7505,2024,9,jaheim bell_2024_9,jaheim bell,jaheim bell,2500,4000


In [120]:
# Compare fuzzy similarity between merge_name_main and merge_name_salary
# Note: key_match should = rapidfuzz so rapidfuzz not necessary but good for verification
# output: comparison of the two methods

test_matches['fuzzy_ratio'] = test_matches.apply(
    lambda row: fuzz.token_sort_ratio(row['merge_name_main'], row['merge_name_salary']),
    axis=1
)

# Check exact vs fuzzy
exact_matches = (test_matches['merge_name_main'] == test_matches['merge_name_salary']).sum()
fuzzy_matches_90plus = (test_matches['fuzzy_ratio'] >= 90).sum()

print(f"🎯 Exact name matches: {exact_matches:,} / {len(test_matches):,}")
print(f"👌 Fuzzy matches ≥ 90: {fuzzy_matches_90plus:,} / {len(test_matches):,}")

🎯 Exact name matches: 8,064 / 8,064
👌 Fuzzy matches ≥ 90: 8,064 / 8,064


In [121]:
# *** Final Merge ***
# output: complete merge of nfl python stats, fantasypros, sportsoddshistory, and bigdataball salary data

# Merge salary data into baseline
te_nfl_py_fp_odds_salary_merged = te_nfl_py_fp_odds_merged.merge(
    te_fd_dk_salary_2017_current_df[['season', 'week', 'merge_key', 'dk_salary', 'fd_salary']],
    on=['season', 'week', 'merge_key'],
    how='left'
)

# Summary match stats
num_salary_matches = te_nfl_py_fp_odds_salary_merged['dk_salary'].notna().sum()
total_rows = len(te_nfl_py_fp_odds_salary_merged)
print(f"💰 Salaries merged into: {num_salary_matches:,} / {total_rows:,} rows ({(num_salary_matches / total_rows) * 100:.2f}%)")

# Optional: remove merge_key if you're done with it
# te_nfl_py_fp_odds_salary_merged.drop(columns=['merge_key'], inplace=True)

# ** csv file **
# te_nfl_py_fp_odds_salary_merged.to_csv("te_nfl_py_fp_odds_salary_merged.csv", index=False)
# print("📁 Saved as: te_nfl_py_fp_odds_salary_merged.csv")

💰 Salaries merged into: 8,064 / 8,825 rows (91.38%)


In [122]:
### End: Data Normalization and Merge Process ###

In [123]:
### Begin: Feature Engineering ###

In [124]:
# Feature engineering is next
# drop unnecessary columns
# output: updated dataframe and csv file with modified columns

# List of columns to drop
cols_to_drop = [
    'player_id', 'player_name', 'position_group', 'mfl_id', 'sportradar_id',
    'fantasypros_id', 'pff_id', 'sleeper_id', 'nfl_id', 'espn_id', 'yahoo_id',
    'fleaflicker_id', 'cbs_id', 'pfr_id', 'cfbref_id', 'rotowire_id',
    'rotoworld_id', 'ktc_id', 'stats_id', 'stats_global_id', 'fantasy_data_id',
    'swish_id', 'merge_name', 'player_gsis_id', 'player_first_name',
    'player_last_name', 'player_short_name', 'game_type', 'team', 'opponent',
    'pfr_player_name', 'pfr_player_id', 'player', 'team_abbr_y', 'merge_key'
]

# Initial state
initial_shape = te_nfl_py_fp_odds_salary_merged.shape
initial_col_count = initial_shape[1]

print("Before modification:")
print(f"Shape: {initial_shape}")
print(f"Total Columns: {initial_col_count}")

# Drop only existing columns
cols_existing = [col for col in cols_to_drop if col in te_nfl_py_fp_odds_salary_merged.columns]
te_nfl_py_fp_odds_salary_merged_mod_cols = te_nfl_py_fp_odds_salary_merged.drop(columns=cols_existing)

# Post-drop validation
final_shape = te_nfl_py_fp_odds_salary_merged_mod_cols.shape
final_col_count = final_shape[1]
dropped_count = len(cols_existing)
expected_final_col_count = initial_col_count - dropped_count

print("\nAfter modification:")
print(f"Shape: {final_shape}")
print(f"Total Columns After Drop: {final_col_count}")
print(f"Number of Columns Dropped: {dropped_count}")
print(f"Expected Final Column Count: {expected_final_col_count}")
print(f"Column Count Validation Passed: {final_col_count == expected_final_col_count}")

# Confirm none of the dropped columns remain
all_dropped = all(col not in te_nfl_py_fp_odds_salary_merged_mod_cols.columns for col in cols_to_drop)
print(f"All Specified Columns Successfully Dropped: {all_dropped}")

# ** csv file **
output_csv_path = "te_nfl_py_fp_odds_salary_merged_mod_cols.csv"
# te_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_csv_path, index=False)
# print(f"\nModified dataframe saved to {output_csv_path}")

Before modification:
Shape: (8825, 105)
Total Columns: 105

After modification:
Shape: (8825, 70)
Total Columns After Drop: 70
Number of Columns Dropped: 35
Expected Final Column Count: 70
Column Count Validation Passed: True
All Specified Columns Successfully Dropped: True


In [125]:
# reorder columns

# Define new column order
priority_cols = [
    'season', 'season_type', 'week', 'name', 'position', 'recent_team',
    'fpts', 'dk_salary', 'fd_salary', 'rost',
    'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under'
]

# Capture original state
original_shape = te_nfl_py_fp_odds_salary_merged_mod_cols.shape
original_columns = te_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("Original Shape:", original_shape)
print("Original First 20 Columns:", original_columns[:20])

# Build final column order
remaining_cols = [col for col in original_columns if col not in priority_cols]
final_col_order = priority_cols + remaining_cols

# Reorder the columns
te_nfl_py_fp_odds_salary_merged_mod_cols = te_nfl_py_fp_odds_salary_merged_mod_cols[final_col_order]

# Post-reorder validation
new_shape = te_nfl_py_fp_odds_salary_merged_mod_cols.shape
new_columns = te_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("\nNew Shape:", new_shape)
print("New First 20 Columns:", new_columns[:20])

# Validation Checks
print("\n✅ Shape Integrity:", original_shape == new_shape)
print("✅ Column Count Matches:", len(original_columns) == len(new_columns))
print("✅ All Columns Preserved:", set(original_columns) == set(new_columns))

# ** csv file **
output_reordered_csv = "te_nfl_py_fp_odds_salary_merged_mod_cols_reordered.csv"
# te_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_reordered_csv, index=False)
# print(f"\nReordered dataframe saved to {output_reordered_csv}")

Original Shape: (8825, 70)
Original First 20 Columns: ['season', 'season_type', 'week', 'position', 'recent_team', 'name', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share']

New Shape: (8825, 70)
New First 20 Columns: ['season', 'season_type', 'week', 'name', 'position', 'recent_team', 'fpts', 'dk_salary', 'fd_salary', 'rost', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under', 'receptions', 'targets', 'receiving_yards']

✅ Shape Integrity: True
✅ Column Count Matches: True
✅ All Columns Preserved: True


In [126]:
# Apply Rolling Averages and Aggregates - 3, 5, and 7 week averages
# Output: updated dataframe with aggregates (optional csv file)

# Start from sorted copy of the main DF
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg = (
    te_nfl_py_fp_odds_salary_merged_mod_cols
    .sort_values(by=['name', 'season', 'week'])
    .reset_index(drop=True)
)

# Feature map: full column -> short prefix
feature_map = {
    'targets': 'tgt',
    'receptions': 'rec',
    'receiving_yards': 'rec_yds',
    'receiving_air_yards': 'rec_air_yards',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Apply rolling averages and lag features
for full_col, short in feature_map.items():
    # Group once
    grouped = te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.groupby(['name', 'season'])

    # Rolling averages using apply (preserves group boundaries)
    for window in windows:
        col_name = f"{short}_{window}wk_avg"
        te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[col_name] = grouped[full_col].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean().shift(1)
        ).reset_index(drop=True)

    # Lag feature (1-game lookback)
    lag_col = f"{short}_lag_1"
    te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[lag_col] = grouped[full_col].shift(1).reset_index(drop=True)

# Final integrity check
print("✅ Final shape:", te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.shape)

# ** csv file **
# te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.to_csv(
#     "te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv")

✅ Final shape: (8825, 90)


In [127]:
# Validation tests of aggregates
# output: there should be no aggregates prior to week 4

# Structural Check — No aggregates in first 3 weeks of a season
def check_early_aggregates(df, cols, earliest_week=4):
    early = df[df['week'] < earliest_week]
    violations = early[cols].notna().sum()
    print("🚨 Aggregates present before week", earliest_week)
    print(violations[violations > 0])

# Boundary Check — Rolling aggregates must reset per season
def check_season_boundaries(df, col_prefix):
    errors = []
    for short in col_prefix:
        col_name = f'{short}_3wk_avg'
        season_transitions = df.groupby(['name'])['season'].diff().fillna(0)
        cross_season_rows = df[season_transitions != 0]
        if cross_season_rows[col_name].notna().any():
            errors.append(col_name)
    if errors:
        print("❌ Rolling values leaked across seasons:", errors)
    else:
        print("✅ No cross-season leakage detected.")

# Shape check
def check_shape(df, expected_cols_added):
    print("✅ Final shape:", df.shape)
    print("✅ Final columns:", df.columns[-expected_cols_added:])

# === Apply Checks ===
rolling_cols = [f"{short}_{w}wk_avg" for short in ['tgt', 'rec', 'rec_yds', 'rec_air_yards', 'fpts'] for w in [3, 5, 7]]
lag_cols = [f"{short}_lag_1" for short in ['tgt', 'rec', 'rec_yds', 'rec_air_yards', 'fpts']]

check_early_aggregates(te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, rolling_cols)
check_season_boundaries(te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, ['tgt', 'rec', 'rec_yds', 'rec_air_yards', 'fpts'])
check_shape(te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, expected_cols_added=len(rolling_cols + lag_cols))

🚨 Aggregates present before week 4
Series([], dtype: int64)
✅ No cross-season leakage detected.
✅ Final shape: (8825, 90)
✅ Final columns: Index(['tgt_3wk_avg', 'tgt_5wk_avg', 'tgt_7wk_avg', 'tgt_lag_1', 'rec_3wk_avg', 'rec_5wk_avg', 'rec_7wk_avg', 'rec_lag_1', 'rec_yds_3wk_avg', 'rec_yds_5wk_avg', 'rec_yds_7wk_avg', 'rec_yds_lag_1', 'rec_air_yards_3wk_avg', 'rec_air_yards_5wk_avg', 'rec_air_yards_7wk_avg', 'rec_air_yards_lag_1', 'fpts_3wk_avg', 'fpts_5wk_avg', 'fpts_7wk_avg', 'fpts_lag_1'], dtype='object')


In [128]:
# Add Trend Features (deltas) - recent performance over / under (3wk, 5wk, 7wk) averages
# output: updated dataframe with deltas (optional csv file)

# new dataframe
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend = te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.copy()

# Column map
feature_map = {
    'targets': 'tgt',
    'receptions': 'rec',
    'receiving_yards': 'rec_yds',
    'receiving_air_yards': 'rec_air_yards',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Create delta (deviation from trend) features
for full_col, short in feature_map.items():
    for window in windows:
        avg_col = f"{short}_{window}wk_avg"
        delta_col = f"{short}_{window}wk_delta"
        te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[delta_col] = (
            te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[full_col] -
            te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[avg_col]
        )

# Summary and export
delta_cols = [f"{short}_{w}wk_delta" for short in feature_map.values() for w in windows]
print("✅ Added delta columns:", delta_cols)
print("✅ Final shape:", te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.shape)

# ** csv file **
# te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.to_csv(
#     "te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv")

✅ Added delta columns: ['tgt_3wk_delta', 'tgt_5wk_delta', 'tgt_7wk_delta', 'rec_3wk_delta', 'rec_5wk_delta', 'rec_7wk_delta', 'rec_yds_3wk_delta', 'rec_yds_5wk_delta', 'rec_yds_7wk_delta', 'rec_air_yards_3wk_delta', 'rec_air_yards_5wk_delta', 'rec_air_yards_7wk_delta', 'fpts_3wk_delta', 'fpts_5wk_delta', 'fpts_7wk_delta']
✅ Final shape: (8825, 105)


In [129]:
# Create boolean columns
# output: updated dataframe with booleans (optional csv file)

# Start from the previous trend-enhanced dataframe
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool = te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.copy()

# Define boolean columns as 0/1 integers
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tgt_ge_5'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 5).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tgt_ge_7'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 7).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['rec_ge_5'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['receptions'] >= 5).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['rec_ge_7'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['receptions'] >= 7).astype(int)

te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share_ge_20'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share'] >= 0.2).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share_ge_30'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share'] >= 0.3).astype(int)

te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['over_100_yds'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['receiving_yards'] >= 100).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['double_digit_targets'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 10).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['boom_week'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] >= 20).astype(int)
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['bust_week'] = (te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] < 5).astype(int)

# If 'home' is already boolean, convert to int
te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['is_home_game'] = te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['home'].astype(int)

# Final shape and column check
print("✅ Final shape:", te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.shape)
print("✅ New boolean columns added.")

# ** csv file **
# te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.to_csv(
#     "te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv")

✅ Final shape: (8825, 116)
✅ New boolean columns added.


In [130]:
# *** Dataframe: this can be used as the final dataframe but the filename is long ***

# split the over / under column into two columns: o_u and total
# output: updated dataframe with o_u and total columns (optional csv file) 

# Copy from final boolean-enriched dataframe
te_nfl_py_fp_odds_salary_features = te_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.copy()

# Extract 'O' or 'U' and map to "over"/"under"
te_nfl_py_fp_odds_salary_features['O_U'] = (
    te_nfl_py_fp_odds_salary_features['over_under']
    .str[0]
    .map({'O': 'over', 'U': 'under'})
)

# Extract the numeric total (handles int or float)
te_nfl_py_fp_odds_salary_features['Total'] = (
    te_nfl_py_fp_odds_salary_features['over_under']
    .str.extract(r'(\d+\.?\d*)')[0]
    .astype(float)
)

# Validation
print("✅ Final shape:", te_nfl_py_fp_odds_salary_features.shape)
print("✅ Sample 'O_U' values:", te_nfl_py_fp_odds_salary_features['O_U'].unique())
print("✅ Sample 'Total' values:", te_nfl_py_fp_odds_salary_features['Total'].dropna().unique()[:5])

# ** csv file **
# te_nfl_py_fp_odds_salary_features.to_csv(
#     "te_nfl_py_fp_odds_salary_features.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: te_nfl_py_fp_odds_salary_features.csv")

✅ Final shape: (8825, 118)
✅ Sample 'O_U' values: ['over' 'under' nan]
✅ Sample 'Total' values: [41.5 44.  40.  37.5 41. ]


In [131]:
### End: Feature Engineering ###

In [132]:
### Begin: Final Dataframe ###

In [133]:
### *** Final Dataframe *** ###

# Rename final dataframe
te_nfl_df_sorted_final = te_nfl_py_fp_odds_salary_features.copy()

# **csv file **
te_nfl_df_sorted_final.to_csv(
    "te_nfl_df_sorted_final.csv",
    index=False,
    float_format="%.2f"
)

print("✅ Final dataframe saved as 'te_nfl_df_sorted_final.csv'")
print("✅ Final shape:", te_nfl_df_sorted_final.shape)

✅ Final dataframe saved as 'te_nfl_df_sorted_final.csv'
✅ Final shape: (8825, 118)


In [134]:
### End: Final Dataframe ###

In [135]:
### Begin: Final Dataframe Summary Statistics ###

In [136]:
# Final Dataframe Summary Statistics
# Output: csv output of the dataframe summary statistics on data types and missing values

# Total columns and datatypes
total_cols = te_nfl_df_sorted_final.shape[1]
dtypes_summary = te_nfl_df_sorted_final.dtypes.value_counts().sort_values(ascending=False)
dtypes_percent = (dtypes_summary / total_cols * 100).round(2)

print("🧠 Data Type Distribution (by count and %):")
for dtype, count in dtypes_summary.items():
    print(f"{str(dtype):15} {count:>3} columns  ({dtypes_percent[dtype]:>5.1f}%)")

# Object-type columns
object_cols = te_nfl_df_sorted_final.select_dtypes(include='object').columns.tolist()
print(f"\n🚨 Object-type columns found ({len(object_cols)} total / {total_cols} columns):")
print(object_cols)

# Missing value summary by count and % of rows
row_count = len(te_nfl_df_sorted_final)
na_counts = te_nfl_df_sorted_final.isna().sum()
na_percent = (na_counts / row_count * 100).round(2)
na_summary = pd.DataFrame({'Missing': na_counts, 'Percent': na_percent})
na_summary = na_summary[na_summary['Missing'] > 0].sort_values(by='Percent', ascending=False)

print(f"\n⚠️ Missing Value Summary (non-zero only) — Top {len(na_summary)} columns:")
display(na_summary)

# ** csv file **
na_summary.to_csv("te_df_final_summary_stats.csv")

🧠 Data Type Distribution (by count and %):
float64          53 columns  ( 44.9%)
float32          34 columns  ( 28.8%)
object           12 columns  ( 10.2%)
int64            11 columns  (  9.3%)
int32             6 columns  (  5.1%)
string            1 columns  (  0.8%)
bool              1 columns  (  0.8%)

🚨 Object-type columns found (12 total / 118 columns):
['season_type', 'name', 'position', 'recent_team', 'opponent_abbr', 'role', 'result', 'score', 'over_under', 'player_display_name', 'team_abbr_x', 'O_U']

⚠️ Missing Value Summary (non-zero only) — Top 84 columns:


,Missing,Percent
50+ yds,7709,87.35
g,7709,87.35
40+ yds,7709,87.35
30+ yds,7709,87.35
20+ yds,7709,87.35
...,...,...
wopr,44,0.50
air_yards_share,44,0.50
target_share,44,0.50
receiving_epa,44,0.50


In [137]:
### End: Final Dataframe Summary Statistics ###